In [184]:
#download packages using pip install -r requirements.txt
%pip install -r requirements.txt

  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached MarkupSafe-3.0.2-cp311-cp311-win_amd64.whl.metadata (4.1 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
   ---------------------------------------- 0.0/103.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/103.3 kB ? eta -:--:--
   --- ------------------------------------ 10.2/103.3 kB ? eta -:--:--
   --- ------------------------------------ 10.2/103.3 kB ? eta -:--:--
   --- ------------------------------------ 10.2/103.3 kB ? eta -:--:--
   --------------- ----------------------- 41.0/103.3 kB 196.9 kB/s eta 0:00:01
   --------------------------- ----------- 71.7/103.3 kB 328.6 kB/s eta 0:00:01
   -------------------------------------- 103.3/103.3 kB 426.9 kB/s eta 0:00:00
Usi


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import datetime as dt
import numpy as np
from ortools.sat.python import cp_model
from pandas import read_excel
import ast

In [2]:
info_shifts = {
    "Shifts": ['D1', 'D2', 'D3', 'D4', 'FM', 'GD', 'KOK', 'A1', 'A2', 'A3', 'GA', 'N'],
    "Begintijd": ['07:00', '07:30', '07:30', '08:45', '08:00', '08:00', '15:00', '15:00', '15:30', '18:30', '16:00', '22:45'],
    "Eindtijd": ['15:00', '15:00', '14:00', '12:45', '12:30', '14:00', '19:30', '23:00', '22:30', '22:30', '22:00', '07:00'],
    "Duur": [8, 7.5, 6.5, 4, 4.5, 6, 4.5, 8, 7, 4, 6, 8.25],
    "Maandag": [1,1,1,1,1,1,1,1,1,1,1,1],
    "Dinsdag": [1,1,1,1,1,1,1,1,1,1,1,1],
    "Woensdag": [1,1,1,1,1,1,1,1,1,1,1,1],
    "Donderdag": [1,1,1,0,1,1,1,1,1,1,1,1],
    "Vrijdag": [1,1,1,1,0,1,1,1,1,1,1,1],
    "Zaterdag": [1,1,1,0,0,1,1,1,1,1,1,1],
    "Zondag": [1,1,1,1,0,1,1,1,1,0,1,1],
    "Deskundigheden": [[1,2], [2], [3], [3], [6], [4], [5], [2], [3], [3], [4], [2]]
}

for i in range(len(info_shifts['Shifts'])):
    info_shifts['Begintijd'][i] = dt.datetime.strptime(info_shifts['Begintijd'][i], '%H:%M').time()
    info_shifts['Eindtijd'][i] = dt.datetime.strptime(info_shifts['Eindtijd'][i], '%H:%M').time()
    
df_shifts = pd.DataFrame(info_shifts)

In [3]:
df_onb = pd.read_csv('data/Onbeschikbaarheid_hoevelaken.csv', sep=';')
df_onb['Datum'] = pd.to_datetime(df_onb['Datum beschikbaarheid'], format='%d-%m-%Y').dt.date
df_onb['Beschikbaarheid'] = df_onb['Beschikbaarheid'].fillna('Onbekend')
df_onb

,Team medewerker,Medewerker id,Datum beschikbaarheid,Beschikbaarheid,Beschikbaarheid tijd vanaf,Beschikbaarheid tijd t/m,Datum
0,Afdeling Hoevelaken,3922,19-6-2025,Scholing,13:00,17:00,2025-06-19
1,Afdeling Hoevelaken,4128,19-6-2025,Scholing,09:00,14:30,2025-06-19
2,Afdeling Hoevelaken,7147,19-6-2025,Scholing,09:00,14:30,2025-06-19
3,Afdeling Hoevelaken,7204,19-6-2025,Beschikbaar,00:00,23:59,2025-06-19
4,Afdeling Hoevelaken,599862,19-6-2025,Niet beschikbaar,00:00,23:59,2025-06-19
...,...,...,...,...,...,...,...
1563,Afdeling Hoevelaken,4128,31-12-2025,Niet beschikbaar,00:00,23:59,2025-12-31
1564,Afdeling Hoevelaken,597492,31-12-2025,Niet beschikbaar,00:00,23:59,2025-12-31
1565,Afdeling Hoevelaken,599363,31-12-2025,Niet beschikbaar,00:00,23:59,2025-12-31
1566,Afdeling Hoevelaken,599371,31-12-2025,Niet beschikbaar,00:00,23:59,2025-12-31


In [4]:
df_werknemers = pd.read_excel('data\medewerkers_compleet.xlsx')
df_werknemers

,medewerker_id,medewerker_naam,wensen,max_werkdgn_pw,contracturen,geboortedatum,contract soort,deskundigheid,contact vanaf,contract tm
0,001109-4,Beau Versluis,geen,5.0,27.0,2004-05-19,vaste uren,4.0,2025-06-01,NaT
1,001221-1,Gerrie Balke,geen,4.0,0.0,1963-03-26,oproep,5.0,2024-04-19,NaT
2,002097-3,Hennie van den Brink,alleen nachten,2.0,12.0,1957-09-06,vaste uren,2.0,2024-09-06,2025-09-05
3,002653-1,Romy van Dijkhuizen,alleen weekenden,1.0,0.0,2007-01-30,oproep,3.0,2025-12-22,2025-12-21
4,003888-1,Jasmijn van den Berg,niet plannen,0.0,0.0,2007-03-02,oproep,NaN,2025-03-01,2025-08-31
5,003888-2,Jasmijn van den Berg,alleen vrijdagavond en weekenden,0.0,0.0,2007-03-02,oproep,34.0,2025-05-06,2025-07-11
6,003922-1,Sylvia Feiken,geen,4.0,18.0,1975-02-18,vaste uren,3.0,2025-03-01,2026-02-28
7,003932-1,Daphne Bosman,geen,5.0,28.0,1992-08-17,vaste uren,3.0,2024-08-24,2025-08-23
8,004128-1,Daphne Kraaikamp,geen,5.0,24.0,1983-12-10,vaste uren,3.0,2025-08-01,NaT
9,005068-1,Liya Chaiyasane,vast rooster,4.0,12.0,1972-08-24,vaste uren,6.0,2024-10-01,2025-09-30


In [5]:
# Convert deskundigheid to list of ints, handle NaN safely
lsts = []
for i in range(len(df_werknemers)):
    val = df_werknemers['deskundigheid'][i]
    if pd.isna(val):
        lst = []
    elif val < 10:
        lst = [int(val)]
    else:
        lst = [int(x) for x in str(int(val))]
    lsts.append(lst)

df_werknemers['deskundigheid'] = lsts

# Convert contract geldig van, contract tm, and geboortedatum to datetime
df_werknemers['contract_vanaf'] = pd.to_datetime(df_werknemers['contract vanaf'], format='%d/%m/%Y')
df_werknemers['contract_tm'] = pd.to_datetime(df_werknemers['contract tm'], format='%d/%m/%Y')
#df_werknemers['geboortedatum'] = pd.to_datetime(df_werknemers['geboortedatum'], format='%d/%b', errors='coerce')
# Drop unnecessary columns
df_werknemers = df_werknemers.drop(columns=['contract vanaf', 'contract tm'])

df_werknemers

KeyError: 'contract vanaf'

In [6]:
df_werknemers

# delete all rows where wensen = niet plannen
df_werknemers = df_werknemers[df_werknemers['wensen'] != 'niet plannen']
df_werknemers = df_werknemers.reset_index(drop=True)
df_werknemers

# If contract soort = oproep, set contracturen to 0
df_werknemers.loc[df_werknemers['contract soort'] == 'oproep', 'contracturen'] = 0


# If max_werkdgn_pw is NaN, set to 5
df_werknemers['max_werkdgn_pw'] = df_werknemers['max_werkdgn_pw'].fillna(5)

# If contracturen is -, set to 0
df_werknemers['contracturen'] = df_werknemers['contracturen'].replace('-', 0)

# DELETE REIJNDERS SINCE THIS IS AN ANOMALY
df_werknemers = df_werknemers[df_werknemers['medewerker_naam'] != 'Reijnders']
df_werknemers = df_werknemers.reset_index(drop=True)
df_werknemers

,medewerker_id,medewerker_naam,wensen,max_werkdgn_pw,contracturen,geboortedatum,contract soort,deskundigheid,contact vanaf,contract tm
0,001109-4,Beau Versluis,geen,5.0,27.0,2004-05-19,vaste uren,[4],2025-06-01,NaT
1,001221-1,Gerrie Balke,geen,4.0,0.0,1963-03-26,oproep,[5],2024-04-19,NaT
2,002097-3,Hennie van den Brink,alleen nachten,2.0,12.0,1957-09-06,vaste uren,[2],2024-09-06,2025-09-05
3,002653-1,Romy van Dijkhuizen,alleen weekenden,1.0,0.0,2007-01-30,oproep,[3],2025-12-22,2025-12-21
4,003888-2,Jasmijn van den Berg,alleen vrijdagavond en weekenden,0.0,0.0,2007-03-02,oproep,"[3, 4]",2025-05-06,2025-07-11
5,003922-1,Sylvia Feiken,geen,4.0,18.0,1975-02-18,vaste uren,[3],2025-03-01,2026-02-28
6,003932-1,Daphne Bosman,geen,5.0,28.0,1992-08-17,vaste uren,[3],2024-08-24,2025-08-23
7,004128-1,Daphne Kraaikamp,geen,5.0,24.0,1983-12-10,vaste uren,[3],2025-08-01,NaT
8,005068-1,Liya Chaiyasane,vast rooster,4.0,12.0,1972-08-24,vaste uren,[6],2024-10-01,2025-09-30
9,005448-1,Sam van Maanen-Arntzen,vast rooster,4.0,16.0,1973-09-21,vaste uren,[5],2025-06-01,NaT


In [10]:
# Read sheet 'tabellen' and range T2:AD35 from the Excel file
workers = pd.read_excel('data/data tbv rooster automaat v3.xlsx', sheet_name='Tabellen', usecols='T:AE', skiprows=1)
workers

,medewerker_id,medewerker_naam,wensen,max_werkdgn_pw,contracturen,geboortedatum,contract soort,functie,deskundigheid.1,datum indienst,datum uit dienst,nachten
0,001109-4,Beau Versluis,geen,5,27,2004-05-19,vaste uren,4. Gastvrouw/Gastheer,4.0,2025-06-01,NaT,niet
1,001221-1,Gerrie Balke,geen,4,0,1963-03-26,oproep,5. Kok,5.0,2024-04-19,NaT,niet
2,002097-3,Hennie van den Brink,alleen nachten,2,12,1957-09-06,vaste uren,2. Verzorgende,2.0,2024-09-06,NaT,uitsluitend
3,002653-1,Romy van Dijkhuizen,alleen weekenden,1,0,2007-01-30,oproep,4. Gastvrouw/Gastheer,4.0,2024-12-22,NaT,niet
4,003888-1,Jasmijn van den Berg,alleen vrijdagavond en weekenden,1,0,2007-03-02,oproep,3. Medewerker woonzorg,34.0,2025-03-01,NaT,niet
5,003888-2,Jasmijn van den Berg,niet plannen,0,0,2007-03-02,oproep,99. NVT,NaN,2025-03-01,2025-08-31,niet
6,003922-1,Sylvia Feiken,geen,4,18,1975-02-18,vaste uren,3. Medewerker woonzorg,3.0,2025-03-01,NaT,niet
7,003932-1,Daphne Bosman,geen,5,28,1992-08-17,vaste uren,3. Medewerker woonzorg,3.0,2024-08-24,NaT,niet
8,004128-1,Daphne Kraaikamp,geen,4,24,1983-12-10,vaste uren,3. Medewerker woonzorg,3.0,2024-08-01,NaT,niet
9,005068-1,Liya Chaiyasane,vast rooster,4,12,1972-08-24,vaste uren,6. Facilitair medewerker,6.0,2024-10-01,NaT,niet


In [7]:
# Delete all rows where shifts = KOK or FM
df_shifts = df_shifts[~df_shifts['Shifts'].isin(['KOK', 'FM'])]

# Convert to long format: each row = one shift on one day
days = ['Maandag', 'Dinsdag', 'Woensdag', 'Donderdag', 'Vrijdag', 'Zaterdag', 'Zondag']
day_map = {day: i for i, day in enumerate(days)}  # Map day names to day indices (0–6)

shift_requirements = []

for idx, row in df_shifts.iterrows():
    for day in days:
        if row[day] == 1:  # If this shift needs to be filled on this day
            shift_requirements.append({
                "shift_name": row['Shifts'],
                "day_of_week": day_map[day],  # 0=Monday, etc.
                "start_time": row['Begintijd'],
                "end_time": row['Eindtijd'],
                "qualification": row['Deskundigheden'],
                "duration": row['Duur']
            })

all_shift_instances = []

for week in range(4):  # For 4 weeks
    week_num = week + 1  # Weeks are 1-indexed
    for row in shift_requirements:
        all_shift_instances.append({
            **row,
            "week": week_num,
            "absolute_day": week * 7 + row["day_of_week"],  # 0–27
        })

shift_instances_df = pd.DataFrame(all_shift_instances)

## Preprocessing function

In [142]:

def preprocess_data(df_shifts: pd.DataFrame, df_werknemers: pd.DataFrame, df_onb: pd.DataFrame, start_date: dt.date = dt.date(2025, 9, 1), num_weeks: int = 4):
    """
    Prepares shifts and workers dataframes for the OR-Tools scheduling model.
    Returns processed shifts, workers, and helper structures.
    """

    # --- Shifts ---
    # Create clean shifts dataframe
    # Delete all rows where shifts = KOK or FM
    df_shifts = df_shifts[~df_shifts['Shifts'].isin(['KOK', 'FM'])]

    # Convert to long format: each row = one shift on one day
    days = ['Maandag', 'Dinsdag', 'Woensdag', 'Donderdag', 'Vrijdag', 'Zaterdag', 'Zondag']
    day_map = {day: i for i, day in enumerate(days)}  # Map day names to day indices (0–6)

    shift_requirements = []

    for idx, row in df_shifts.iterrows():
        for day in days:
            if row[day] == 1:  # If this shift needs to be filled on this day
                shift_requirements.append({
                    "shift_name": row['Shifts'],
                    "day_of_week": day_map[day],  # 0=Monday, etc.
                    "start_time": row['Begintijd'],
                    "end_time": row['Eindtijd'],
                    "qualification": row['Deskundigheden'],
                    "duration": row['Duur']
                })

    all_shift_instances = []

    
    for week in range(num_weeks):
        for row in shift_requirements:
            # Compute shift_date based on start_date
            shift_date = start_date + dt.timedelta(days=week*7 + row["day_of_week"])
            all_shift_instances.append({
                **row,
                "week": week + 1,
                "absolute_day": week * 7 + row["day_of_week"],  # 0-based absolute day
                "shift_date": shift_date
            })

    shifts = pd.DataFrame(all_shift_instances)
    shifts['shift_id'] = shifts.index.astype(int)

    # Convert durations (hours → minutes, integers for CP-SAT)
    shifts['duration_min'] = (shifts['duration'].astype(float) * 60).round().astype(int)

    # Ensure int types
    shifts['week'] = shifts['week'].astype(int)
    shifts['day_of_week'] = shifts['day_of_week'].astype(int)

    # Day key for grouping
    shifts['day_key'] = list(zip(shifts['week'], shifts['day_of_week']))

    # add is_night column based on shift_name (e.g., 'N' is night)
    shifts['is_night'] = shifts['shift_name'].apply(lambda x: 1 if x == 'N' else 0)
    shifts['is_night'] = shifts['is_night'].astype(bool)
    shifts['shift_id'] = shifts.index.astype(int)
    
    shifts['qualification'] = shifts['qualification'].apply(
        lambda q: ast.literal_eval(q) if isinstance(q, str) else q)
    
    shifts = shifts[['shift_id', 'shift_name', 'shift_date', 'week', 'day_of_week', 'absolute_day', 'start_time', 'end_time', 'duration_min', 'qualification', 'is_night', 'day_key']]
    
    # --- Workers ---
    workers = df_werknemers.copy().reset_index(drop=True)
    workers['medewerker_id'] = workers['medewerker_id'].astype(str)
    # DELETE REIJNDERS SINCE THIS IS AN ANOMALY
    workers = workers[workers['medewerker_naam'] != 'Reijnders']
    workers = workers.reset_index(drop=True)
    # Change werknemers_id to delete every 0 at the start and everything and including the first -
    workers['medewerker_id'] = workers['medewerker_id'].apply(lambda x: x.lstrip('0').split('-')[0])
    
    # delete all rows where wensen = niet plannen
    workers = workers[workers['wensen'] != 'niet plannen']
    workers = workers.reset_index(drop=True)
    
    # Convert contract geldig van, contract tm, and geboortedatum to datetime
    workers['contract_vanaf'] = pd.to_datetime(workers['contact vanaf'], format='%d/%m/%Y')
    workers['contract_tm'] = pd.to_datetime(workers['contract tm'], format='%d/%m/%Y')
    #If contract_tm is NaT, set to 31-12-2099
    workers['contract_tm'] = workers['contract_tm'].fillna(pd.Timestamp('2099-12-31'))
    
    # Drop unnecessary columns
    workers = workers.drop(columns=['contact vanaf', 'contract tm'])
    
    #Convert deskundigheid to list of ints
    lsts = []
    for i in range(len(workers)):
        lst = []
        if workers['deskundigheid'][i] <10:
            lst = [int(workers['deskundigheid'][i])]
        else:
            lst = [int(x) for x in str(int(workers['deskundigheid'][i]))]
        
        lsts.append(lst)
        
    workers['deskundigheid'] = lsts

    workers['deskundigheid'] = workers['deskundigheid'].apply(
        lambda q: ast.literal_eval(q) if isinstance(q, str) else q
    )
    # Delete all employees with deskundigheid = 5 or 6
    workers = workers[~workers['deskundigheid'].apply(lambda x: isinstance(x, (list, tuple)) and (5 in x or 6 in x or 7 in x))]
    
    # If contract soort = oproep, set contracturen to 0
    workers.loc[workers['contract soort'] == 'oproep', 'contracturen'] = 0
    
    # Normalize contract details
    workers['max_days_per_week'] = workers['max_werkdgn_pw'].fillna(0).astype(int)
    # Safely convert contracturen to float, fill NaN with 0 before converting to int
    workers['contract_hours'] = workers['contracturen'].fillna(0).astype(float)
    workers['contract_minutes'] = (workers['contract_hours'] * 60).round().astype(int) 
    # Convert geboortedatum to leeftijd based on current day (round down if birthday not yet occurred this year)
    current_date = dt.datetime.now().date()

    workers['leeftijd'] = workers['geboortedatum'].apply(
        lambda x: current_date.year - x.year - ((current_date.month, current_date.day) < (x.month, x.day))
    )
    
    # Create IDs
    emp_ids = workers['medewerker_id'].tolist()
    #emp_ids = [str(e) for e in emp_ids]
    emp_index = {emp: i for i, emp in enumerate(emp_ids)}

    # --- onhbeschikbaarheid ---
    # Convert 'Datum beschikbaarheid' to datetime
    df_onb['Datum'] = pd.to_datetime(df_onb['Datum beschikbaarheid'], format='%d-%m-%Y').dt.date
    df_onb['Beschikbaarheid'] = df_onb['Beschikbaarheid'].fillna('Onbekend')
    df_onb['Beschikbaarheid_tijd_vanaf'] = pd.to_datetime(df_onb['Beschikbaarheid tijd vanaf'], format='%H:%M', errors='coerce').dt.time
    df_onb['Beschikbaarheid_tijd_tm'] = pd.to_datetime(df_onb['Beschikbaarheid tijd t/m'], format='%H:%M', errors='coerce').dt.time
    df_onb = df_onb[['Medewerker id', 'Datum', 'Beschikbaarheid', 'Beschikbaarheid_tijd_vanaf', 'Beschikbaarheid_tijd_tm']]
    df_onb['Medewerker id'] = df_onb['Medewerker id'].astype(str)

    # --- Helper dictionaries ---
    dur_min = shifts.set_index('shift_id')['duration_min'].to_dict()

    # Grouping by week/day
    shifts_by_week = {}
    #shifts_by_day = {}
    for _, r in shifts.iterrows():
        s = int(r['shift_id'])
        w = int(r['week'])
        day_k = r['day_key']
        shifts_by_week.setdefault(w, []).append(s)
        #shifts_by_day.setdefault(day_k, []).append(s)

    shifts_by_day = shifts.groupby('absolute_day')['shift_id'].apply(list).to_dict()
    
    weeks = sorted(shifts['week'].unique().tolist())
    
    night_shifts = shifts.loc[shifts['is_night'], 'shift_id'].tolist()
    night_shifts_by_week = {
        w: shifts.loc[(shifts['week'] == w) & (shifts['is_night']), 'shift_id'].tolist() for w in weeks}

    return {
        "shifts": shifts,
        "workers": workers,
        "onb": df_onb,
        "emp_ids": emp_ids,
        "emp_index": emp_index,
        "dur_min": dur_min,
        "shifts_by_week": shifts_by_week,
        "shifts_by_day": shifts_by_day,
        "weeks": weeks,
        "night_shifts": night_shifts,
        "night_shifts_by_week": night_shifts_by_week
    }

In [144]:
info_shifts = {
    "Shifts": ['D1', 'D2', 'D3', 'D4', 'FM', 'GD', 'KOK', 'A1', 'A2', 'A3', 'GA', 'N'],
    "Begintijd": ['07:00', '07:30', '07:30', '08:45', '08:00', '08:00', '15:00', '15:00', '15:30', '18:30', '16:00', '22:45'],
    "Eindtijd": ['15:00', '15:00', '14:00', '12:45', '12:30', '14:00', '19:30', '23:00', '22:30', '22:30', '22:00', '07:00'],
    "Duur": [8, 7.5, 6.5, 4, 4.5, 6, 4.5, 8, 7, 4, 6, 8.25],
    "Maandag": [1,1,1,1,1,1,1,1,1,1,1,1],
    "Dinsdag": [1,1,1,1,1,1,1,1,1,1,1,1],
    "Woensdag": [1,1,1,1,1,1,1,1,1,1,1,1],
    "Donderdag": [1,1,1,0,1,1,1,1,1,1,1,1],
    "Vrijdag": [1,1,1,1,0,1,1,1,1,1,1,1],
    "Zaterdag": [1,1,1,0,0,1,1,1,1,1,1,1],
    "Zondag": [1,1,1,1,0,1,1,1,1,0,1,1],
    "Deskundigheden": [[1,2], [2], [3], [3], [6], [4], [5], [2], [3], [3], [4], [2]]
}

for i in range(len(info_shifts['Shifts'])):
    info_shifts['Begintijd'][i] = dt.datetime.strptime(info_shifts['Begintijd'][i], '%H:%M').time()
    info_shifts['Eindtijd'][i] = dt.datetime.strptime(info_shifts['Eindtijd'][i], '%H:%M').time()
    
df_shifts = pd.DataFrame(info_shifts)

df_werknemers = pd.read_excel('data\medewerkers_compleet.xlsx')

df_onb = pd.read_csv('data/Onbeschikbaarheid_hoevelaken.csv', sep=';')
df_onb['Datum'] = pd.to_datetime(df_onb['Datum beschikbaarheid'], format='%d-%m-%Y').dt.date
df_onb['Beschikbaarheid'] = df_onb['Beschikbaarheid'].fillna('Onbekend')

preprocessed_data = preprocess_data(df_shifts, df_werknemers, df_onb, start_date = dt.date(2025, 9, 1))
preprocessed_data['emp_ids']

C:\Users\20192507\AppData\Local\Temp\ipykernel_18176\2920187816.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_onb['Medewerker id'] = df_onb['Medewerker id'].astype(str)


['1109',
 '2097',
 '2653',
 '3888',
 '3922',
 '3932',
 '4128',
 '7204',
 '597492',
 '599354',
 '599363',
 '599862',
 '601011',
 '602056',
 '602725',
 '602859',
 '603722',
 '605162',
 '627311',
 '632325',
 '640971']

# Auto-Rooster

In [ ]:
def auto_rooster(data, time_limit_s=60):
    shifts = data['shifts']
    workers = data['workers']
    onb = data['onb']
    emp_ids = data['emp_ids']
    dur_min = data['dur_min']
    shifts_by_week = data['shifts_by_week']
    shifts_by_day = data['shifts_by_day']
    weeks = data['weeks']
    night_shifts = data['night_shifts']
    night_shifts_by_week = data['night_shifts_by_week']
    
    num_weeks = len(weeks)
    
    CAO_7_1_exempt = set(map(str, [602859])) # Teuna
    CAO_7_4_exempt = set(map(str, [2097, 602859])) # Hennie and Teuna

    # --- helper maps once 
    days_list = sorted(shifts['absolute_day'].unique().tolist())
    night_shifts_by_day = shifts[shifts['is_night']].groupby('absolute_day')['shift_id'].apply(list).to_dict()
    
    print("===STARTING SCHEDULING===")
    print(f"Number of shifts: {len(shifts)}")
    print(f"Number of employees: {len(emp_ids)}")
    print(f"Number of weeks: {num_weeks}")
    print(f"Total working hours employees: {workers['contract_hours'].sum() * 4:.1f}")
    print(f"Total required shift hours: {shifts['duration_min'].sum() / 60:.1f}")
    print(f"Difference (hours): {workers['contract_hours'].sum() * 4 - shifts['duration_min'].sum() / 60:.1f}")
    print("=========================")
    
    # --- Model ---    
    model = cp_model.CpModel()
    
    ### Variables ###
    # Decision variables
    x = {(s, emp): model.NewBoolVar(f"x_s{s}_e{emp}")
         for s in shifts['shift_id'] for emp in emp_ids}

    # Shift not covered variable to allow max coverage
    u = {s: model.NewBoolVar(f"uncovered_s{s}") for s in shifts['shift_id']}

    # Night-indicator BoolVars per employee per day: n_emp_day = 1 iff emp works night shift that day
    n_emp_day = {}  # (emp, day) -> BoolVar
    for emp in emp_ids:
        for day in days_list:
            b = model.NewBoolVar(f"n_emp{emp}_day{day}")
            n_emp_day[(emp, day)] = b
            night_ids = night_shifts_by_day.get(day, [])
            if night_ids:
                # If b == 1  => sum(x[night_ids]) >= 1
                model.Add(sum(x[(s, emp)] for s in night_ids) >= 1).OnlyEnforceIf(b)
                # If b == 0  => sum(x[night_ids]) == 0
                model.Add(sum(x[(s, emp)] for s in night_ids) == 0).OnlyEnforceIf(b.Not())
            else:
                # no night shifts possible that day -> force b = 0
                model.Add(b == 0)
                
    ### Constraints ###
    ## Coverage and basic contract constraints
    # 1) Coverage constraint: sum of x + uncovered == 1
    for s in shifts['shift_id']:
        model.Add(sum(x[(s, emp)] for emp in emp_ids) + u[s] == 1)

    # 2) At most one shift per employee per day
    for emp in emp_ids:
        for day_k, s_list in shifts_by_day.items():
            model.AddAtMostOne(x[(s, emp)] for s in s_list)
    
    # 3) After night shift, no day or evening shift the next day. but night shifts are allowed
    for emp in emp_ids:
        for day in shifts['absolute_day'].unique():
            # Night shifts on this day
            night_shifts_today = shifts.loc[(shifts['absolute_day'] == day) & (shifts['is_night']), 'shift_id'].tolist()
            if not night_shifts_today:
                continue
            
            # allow next day night shifts, but no day or evening shifts
            next_day_shifts = shifts.loc[(shifts['absolute_day'] == day + 1) & (~shifts['is_night']), 'shift_id'].tolist()
            if not next_day_shifts:
                continue
            
            # Create a combined constraint:
            # If any night shift is worked on day D, no shift can be worked on day D+1
            model.Add(
                sum(x[(ns, emp)] for ns in night_shifts_today) +
                sum(x[(bs, emp)] for bs in next_day_shifts)
                <= len(night_shifts_today)
            )

    # 4) Max work days per week
    for emp in emp_ids:
        max_days = int(workers.loc[workers['medewerker_id'] == emp, 'max_days_per_week'].iloc[0])
        for w in weeks:
            s_list = shifts_by_week.get(w, [])
            if s_list:
                model.Add(sum(x[(s, emp)] for s in s_list) <= max_days)
    
    # 5) Contract hours on average across all weeks (total contract minutes ≤ weekly cap * number of weeks)
    for emp in emp_ids:
        cap_minutes = int(workers.loc[workers['medewerker_id'] == emp, 'contract_minutes'].iloc[0])
        total_shifts = [s for s in shifts['shift_id']]  # all shifts in planning horizon
        model.Add(sum(dur_min[s] * x[(s, emp)] for s in total_shifts) <= cap_minutes * num_weeks)
    
    # 6) Respect unavailable times
    for _, r in onb.iterrows():
        emp = r['Medewerker id']
        if emp not in emp_ids:
            continue

        date_unb = r['Datum']
        besch = r['Beschikbaarheid'].lower()
        
        # If fully available, skip
        if besch in ['beschikbaar']:
            continue

        # Get unavailability times for that day
        start_unb = r.get('Beschikbaarheid_tijd_vanaf')
        end_unb = r.get('Beschikbaarheid_tijd_tm')
        
        for _, sr in shifts.iterrows():
            shift_date = sr['shift_date']
            if shift_date != date_unb:
                continue
            
            # Check if the shift overlaps with unavailability
            shift_start = sr['start_time']
            shift_end = sr['end_time']
            
            # Overlap condition
            overlap = not (shift_end <= start_unb or shift_start >= end_unb)
            if overlap:
                s = int(sr['shift_id'])
                model.Add(x[(s, emp)] == 0)
    
    ## CAO night shift constraints
    # 7.1) Max 5 consecutive nights
    for emp in emp_ids:
        # Get all night shifts for this employee
        night_shift_ids = [s for s in night_shifts if s in x]  # Only include available ones
        # Sort night shifts by absolute_day to detect sequences
        nights_sorted = sorted(night_shift_ids, key=lambda s: shifts.loc[shifts['shift_id']==s, 'absolute_day'].iloc[0])
        
        # employees who have chosen to work max 7 nights
        if emp in CAO_7_1_exempt:
            max_consec = 7
        else:
            max_consec = 5
        
        for i in range(len(nights_sorted) - max_consec):
            # Window of 6 or 8 consecutive nights
            window = nights_sorted[i:i+max_consec+1]
            #max_consec = 5
            #model.Add(sum(x[(s, emp)] for s in window) <= max_consec)
            
            days_window = [int(shifts.loc[shifts['shift_id'] == s, 'absolute_day'].iloc[0]) for s in window]
            # only enforce if these 6 night-shifts are on 6 consecutive days
            if days_window[-1] - days_window[0] == max_consec:
                model.Add(sum(x[(s, emp)] for s in window) <= max_consec)
    
    # 7.2) After ≥3 consecutive nights → 46h rest (next 2 days off)
    # Detect a block end: days (d-2, d-1, d) are all nights AND (d+1) is NOT a night => that's an end-of-block
    # For each such possible end day `d`, forbid any shift on d+1 and d+2 when the condition holds.
    for emp in emp_ids:
        for idx in range(2, len(days_list)):             # idx corresponds to day d at days_list[idx]
            d2 = days_list[idx]   # the candidate "end" day (third of triple)
            d1 = days_list[idx-1]
            d0 = days_list[idx-2]
            # condition literals: n(emp,d0) & n(emp,d1) & n(emp,d2) & not n(emp,d2+1)
            # require d2+1 to exist in planning horizon to test next-day not-night; if it doesn't exist, skip enforcement
            if (d2 + 1) not in days_list:
                continue
            # blocked days are d2+1 and d2+2 (d2+2 may or may not exist; we will handle existing ones)
            blocked_days = [d2 + 1, d2 + 2]
            # build literal list for OnlyEnforceIf: [n_d0, n_d1, n_d2, n_{d2+1}.Not()]
            cond_literals = [
                n_emp_day[(emp, d0)],
                n_emp_day[(emp, d1)],
                n_emp_day[(emp, d2)],
                n_emp_day[(emp, d2 + 1)].Not()
            ]

            # collect blocked shift ids that actually exist in our horizon
            blocked_shift_ids = []
            for bd in blocked_days:
                blocked_shift_ids.extend(shifts_by_day.get(bd, []))

            if not blocked_shift_ids:
                continue

            # enforce: when the condition holds (i.e. the triple is the end of a night-block),
            # then every blocked shift must be 0 (no assignment).
            for bs in blocked_shift_ids:
                model.Add(x[(bs, emp)] == 0).OnlyEnforceIf(cond_literals)

    # 7.3) Max 35 nights per 13 weeks
    for emp in emp_ids:
        for start_week in range(min(weeks), max(weeks)-12):
            window_weeks = list(range(start_week, start_week+13))
            window_shifts = [s for w in window_weeks for s in night_shifts_by_week.get(w, [])]
            if window_shifts:
                model.Add(sum(x[(s, emp)] for s in window_shifts) <= 35)
    
    # 7.4) Employee older that 55: no night shifts
    for emp in emp_ids:
        # Exclude employees who have chosen to work nights
        if emp in CAO_7_4_exempt:
            continue
        
        leeftijd = int(workers.loc[workers['medewerker_id'] == emp, 'leeftijd'].iloc[0])
        if leeftijd > 55:
            for s in night_shifts:
                model.Add(x[(s, emp)] == 0)
    
    # 8) Qualification match: only assign employees to shifts they are qualified for    
    for _, shift_row in shifts.iterrows():
        sid = int(shift_row['shift_id'])
        required_quals = shift_row['qualification']  # could be int or list

        # Make sure required_quals is a list
        if not isinstance(required_quals, list):
            required_quals = [required_quals]

        for emp in emp_ids:
            emp_quals = workers.loc[workers['medewerker_id'] == emp, 'deskundigheid'].iloc[0]
            if not any(q in emp_quals for q in required_quals):
                # Employee does NOT have any of the required qualifications → forbid assignment
                model.Add(x[(sid, emp)] == 0)
    
    ## Wensen constraints
    
    # 9.1) Hennie van den Brink (2097): Alleen nachtdiensten
    emp_vdBrink = '2097'
    if emp_vdBrink in emp_ids:
        for s in shifts['shift_id']:
            if s not in night_shifts:
                model.Add(x[(s, emp_vdBrink)] == 0)
                
    # 9.2) Romy van Dijkhuizen (2653): Alleen weekenden
    emp_vDijkhuizen = '2653'
    if emp_vDijkhuizen in emp_ids:
        for _, r in shifts.iterrows():
            s = int(r['shift_id'])
            day_of_week = int(r['day_of_week'])
            if day_of_week not in [5, 6]:
                model.Add(x[(s, emp_vDijkhuizen)] == 0)
    # 9.3) Jasmijn van den Berg (3888): Alleen vrijdagavond en weekend
    emp_vdBerg = '3888'
    if emp_vdBerg in emp_ids:
        for _, r in shifts.iterrows():
            s = int(r['shift_id'])
            day_of_week = int(r['day_of_week'])
            shift_name = r['shift_name']
            # Friday evening (day_of_week == 4 and 'A' in shift_name), or Saturday/Sunday (5, 6)
            if not ((day_of_week == 4 and 'A' in shift_name) or day_of_week == 5 or day_of_week == 6):
                model.Add(x[(s, emp_vdBerg)] == 0)
    
    # 9.4) Maike van Kasbergen (601011): Maandag, dinsdag, woensdag geen diensten
    emp_vKasbergen = '601011'
    if emp_vKasbergen in emp_ids:
        for _, r in shifts.iterrows():
            s = int(r['shift_id'])
            day_of_week = int(r['day_of_week'])
            if day_of_week in [0, 1, 2]:
                model.Add(x[(s, emp_vKasbergen)] == 0)
    
    # 9.5) Marlies Schreurs (603722): Max 2 days in a row, and after a block of shifts, always 2 days off
    emp_Schreurs = '603722'
    if emp_Schreurs in emp_ids:
        # create day-level BoolVars linked to actual shift assignments for Marlies
        work_day = {}   # (emp, abs_day) -> BoolVar
        for day in days_list:
            w = model.NewBoolVar(f"work_{emp_Schreurs}_day{day}")
            work_day[day] = w
            shifts_today = shifts_by_day.get(day, [])
            if shifts_today:
                # If w == 1 then at least one shift that day is assigned to Marlies
                model.Add(sum(x[(s, emp_Schreurs)] for s in shifts_today) >= 1).OnlyEnforceIf(w)
                # If w == 0 then no shift that day is assigned to Marlies
                model.Add(sum(x[(s, emp_Schreurs)] for s in shifts_today) == 0).OnlyEnforceIf(w.Not())
            else:
                # no shifts that day -> cannot work
                model.Add(w == 0)

        # 1) Max 2 days in a row: for every triple (d, d+1, d+2) enforce sum <= 2
        for idx in range(0, len(days_list) - 2):
            d = days_list[idx]
            d1 = days_list[idx + 1]
            d2 = days_list[idx + 2]
            # Only apply when all those days exist in horizon (they do, because we indexed)
            model.Add(work_day[d] + work_day[d1] + work_day[d2] <= 2)

        # 2) Enforce 2 days off after any block end:
        # If Marlies works on day d and does NOT work on day d+1, then day d+2 must be off.
        # (For the case she works d AND d+1, the triple-window (above) already forces d+2==0.)
        for idx in range(0, len(days_list) - 2):
            d = days_list[idx]
            d1 = days_list[idx + 1]
            d2 = days_list[idx + 2]
            # If works today and NOT tomorrow -> day after tomorrow must be 0
            model.Add(work_day[d2] == 0).OnlyEnforceIf([work_day[d], work_day[d1].Not()])
    
    # 9.6) Teuna de Groot (602859): 7 night shifts in a row, 7 days off in a row pattern
    emp_deGroot = '602859'
    if emp_deGroot in emp_ids:
        # 1) forbid any non-night shift for Teuna
        for s in shifts['shift_id']:
            if s not in night_shifts:
                model.Add(x[(s, emp_deGroot)] == 0)

        # 2) create day-level BoolVars linked to actual night assignments for Teuna
        night_work_day = {}  # (day) -> BoolVar (1 if Teuna works a night shift that day)
        for day in days_list:
            b = model.NewBoolVar(f"teuna_night_{day}")
            night_work_day[day] = b
            shifts_today = shifts_by_day.get(day, [])
            night_shifts_today = [s for s in shifts_today if bool(shifts.loc[shifts['shift_id'] == s, 'is_night'].iloc[0])]

            if night_shifts_today:
                # b == 1  <=> at least one night shift assigned that day
                model.Add(sum(x[(s, emp_deGroot)] for s in night_shifts_today) >= 1).OnlyEnforceIf(b)
                model.Add(sum(x[(s, emp_deGroot)] for s in night_shifts_today) == 0).OnlyEnforceIf(b.Not())
            else:
                model.Add(b == 0)

        # 3) Enforce strict repeating 7-on / 7-off pattern with a 1-of-14 offset (phase)
        #    Pick one offset k in 0..13. If offset k is chosen then
        #      for each day d: night_work_day[d] == 1 iff (d - base - k) % 14 < 7
        period = 14
        base_day = days_list[0]  # reference
        # If planning horizon shorter than one full period, fallback to "no partial night blocks"
        if len(days_list) >= period:
            offsets = []
            for k in range(period):
                vk = model.NewBoolVar(f"teuna_phase_{k}")
                offsets.append(vk)
            # exactly one phase selected
            model.Add(sum(offsets) == 1)

            # link each (day, phase) to expected value
            for k, vk in enumerate(offsets):
                for d in days_list:
                    pos = (d - base_day - k) % period
                    if pos < 7:
                        # day d is in the 7-on part for phase k => must be 1 if vk is chosen
                        model.Add(night_work_day[d] == 1).OnlyEnforceIf(vk)
                    else:
                        # day d is in the 7-off part for phase k => must be 0 if vk is chosen
                        model.Add(night_work_day[d] == 0).OnlyEnforceIf(vk)
                        
    # 9.8) Carolien van de Koppel (602056): Max 3 shifts per week and only late shifts (A or N)
    emp_vdKoppel = '602056'
    if emp_vdKoppel in emp_ids:
        for w in weeks:
            s_list = shifts_by_week.get(w, [])
            if s_list:
                model.Add(sum(x[(s, emp_vdKoppel)] for s in s_list) <= 3)
                for s in s_list:
                    shift_name = shifts.loc[shifts['shift_id'] == s, 'shift_name'].iloc[0]
                    if 'A' not in shift_name and 'N' not in shift_name:
                        model.Add(x[(s, emp_vdKoppel)] == 0)

                
    ### Objective ###
    # Objective: minimize uncovered shifts
    penalty_uncovered = 1  # weight for uncovered shifts

    model.Minimize(
        penalty_uncovered * sum(u.values())
        )

    ### Solve ###
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = time_limit_s
    solver.parameters.num_search_workers = 8
    status = solver.Solve(model)

    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        assignments = []
        uncovered = []
    
        for _, r in shifts.iterrows():
            sid = int(r['shift_id'])
            assigned = False
            
            for emp in emp_ids:
                if solver.Value(x[(sid, emp)]) == 1:
                    assignments.append({
                        'shift_id': sid,
                        'shift_name': r['shift_name'],
                        'start_time': r['start_time'],
                        'end_time': r['end_time'],
                        'shift_date': r['shift_date'],
                        'is_night': r['is_night'],
                        'week': int(r['week']),
                        'day_of_week': int(r['day_of_week']),
                        'absolute_day': int(r['absolute_day']),
                        'duration_min': int(r['duration_min']),
                        'employee_id': emp,
                        'employee_name': workers.loc[workers['medewerker_id'] == emp, 'medewerker_naam'].iloc[0],
                        'qualification': r['qualification'],
                        'deskundigheid': workers.loc[workers['medewerker_id'] == emp, 'deskundigheid'].iloc[0]
                    })
                    assigned = True
                    break
            
            # If no employee assigned, mark as uncovered with missing employee info
            if not assigned:
                uncovered.append(sid)
                assignments.append({
                    'shift_id': sid,
                    'shift_name': r['shift_name'],
                    'start_time': r['start_time'],
                    'end_time': r['end_time'],
                    'shift_date': r['shift_date'],
                    'is_night': r['is_night'],
                    'week': int(r['week']),
                    'day_of_week': int(r['day_of_week']),
                    'absolute_day': int(r['absolute_day']),
                    'duration_min': int(r['duration_min']),
                    'employee_id': None,
                    'employee_name': None,
                    'qualification': r['qualification'],
                    'deskundigheid': None
                })
        
        # Calculate penalties explicitly
        uncovered_penalty = len(uncovered)
        
        print("=== Solution Summary ===")
        print(f"  Uncovered shifts: {len(uncovered)} (penalty {uncovered_penalty})")
        
        return {
            "assignments_df": pd.DataFrame(assignments),
            "uncovered_shifts": uncovered,
            "objective_value": solver.ObjectiveValue(),
            "solver_status": solver.StatusName(status)
        }
    else:
        print("No solution found:", solver.StatusName(status))
        return None

In [102]:
result = auto_rooster(preprocessed_data, time_limit_s=120)
result['assignments_df'].to_csv('output/shift_assignments_01_09_2025.csv', index=False)

result['assignments_df']

[1, 8, 15, 22, 27, 34, 41, 48, 54, 61]
===STARTING SCHEDULING===
Number of shifts: 268
Number of employees: 21
Number of weeks: 4
Total working hours employees: 1544.0
Total required shift hours: 1779.0
Difference (hours): -235.0
=== Solution Summary ===
  Uncovered shifts: 42 (penalty 42)


,shift_id,shift_name,start_time,end_time,shift_date,is_night,week,day_of_week,absolute_day,duration_min,employee_id,employee_name,qualification,deskundigheid
0,0,D1,07:00:00,15:00:00,2025-09-01,False,1,0,0,480,599363,Arja van Pelt-Ymker,"[1, 2]",[1]
1,1,D1,07:00:00,15:00:00,2025-09-02,False,1,1,1,480,599363,Arja van Pelt-Ymker,"[1, 2]",[1]
2,2,D1,07:00:00,15:00:00,2025-09-03,False,1,2,2,480,599354,Ali van Soeren,"[1, 2]",[1]
3,3,D1,07:00:00,15:00:00,2025-09-04,False,1,3,3,480,599354,Ali van Soeren,"[1, 2]",[1]
4,4,D1,07:00:00,15:00:00,2025-09-05,False,1,4,4,480,599354,Ali van Soeren,"[1, 2]",[1]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,263,N,22:45:00,07:00:00,2025-09-24,True,4,2,23,495,602859,Teuna de Groot,[2],[2]
264,264,N,22:45:00,07:00:00,2025-09-25,True,4,3,24,495,602859,Teuna de Groot,[2],[2]
265,265,N,22:45:00,07:00:00,2025-09-26,True,4,4,25,495,602859,Teuna de Groot,[2],[2]
266,266,N,22:45:00,07:00:00,2025-09-27,True,4,5,26,495,602725,Marileen van Dam,[2],[2]


# Validation function

In [ ]:
def validate_auto_rooster(assignments_df, shifts, workers, dur_min, onb, night_shifts, night_shifts_by_week):
    errors = []

    # delete all rows in assignments_df where employee_id is NaN
    assignments_df = assignments_df.dropna(subset=['employee_id'])
    
    # Helper: quick lookup
    shifts_lookup = shifts.set_index('shift_id')

    COA_7_1_exempt = set(map(str, [602859])) # Teuna
    CAO_7_4_exempt = set(map(str, [2097, 602859])) # Hennie and Teuna

    ## 1) Coverage: each shift has <= 1 assigned employee
    for sid, group in assignments_df.groupby("shift_id"):
        if len(group) > 1:
            errors.append(f"Shift {sid} assigned to multiple employees: {group['employee_id'].tolist()}")

    ## 2) At most one shift per employee per day
    for (emp, day), group in assignments_df.groupby(["employee_id", "absolute_day"]):
        if len(group) > 1:
            errors.append(f"Employee {emp} works {len(group)} shifts on day {day}: {group['shift_id'].tolist()}")

    ## 3) After night shift, no day/evening shift next day
    for emp, group in assignments_df.groupby("employee_id"):
        nights = group[group["is_night"] == True]
        for _, ns in nights.iterrows():
            next_day = ns['absolute_day'] + 1
            next_day_shifts = group[group['absolute_day'] == next_day]
            #if next_day_shifts is isnight shifts, it's ok
            next_day_shifts = next_day_shifts[next_day_shifts['is_night'] == False]
            
            # Any next day assignment is an error
            if len(next_day_shifts) > 0:
                errors.append(f"Employee {emp} has day/evening shift(s) on day {next_day} after a night shift on day {ns['absolute_day']}.")

    ## 4) Max work days per week
    for emp, group in assignments_df.groupby("employee_id"):
        max_days = int(workers.loc[workers['medewerker_id'] == emp, 'max_days_per_week'].iloc[0])
        for week, week_group in group.groupby("week"):
            worked_days = week_group['absolute_day'].nunique()
            if worked_days > max_days:
                errors.append(f"Employee {emp} exceeds max work days in week {week}: {worked_days} > {max_days}")

    ## 5) Contract hours on average across all weeks
    num_weeks = assignments_df['week'].nunique()

    for emp, group in assignments_df.groupby("employee_id"):
        cap_minutes = int(workers.loc[workers['medewerker_id'] == emp, 'contract_minutes'].iloc[0])
        total_minutes = group['duration_min'].sum()
        allowed_total = cap_minutes * num_weeks

        if total_minutes > allowed_total:
            errors.append(
                f"Employee {emp} exceeds average contract hours: "
                f"{total_minutes} min worked > {allowed_total} min allowed over {num_weeks} weeks"
            )

    ## 6) No shifts overlapping with unavailable times
    for _, r in onb.iterrows():
        emp = r['Medewerker id']
        if emp not in workers['medewerker_id'].values:
            continue

        besch = r['Beschikbaarheid'].lower()
        if besch == 'beschikbaar':
            continue

        date_unb = r['Datum']
        start_unb = r.get('Beschikbaarheid_tijd_vanaf')
        end_unb = r.get('Beschikbaarheid_tijd_tm')

        emp_shifts = assignments_df[assignments_df["employee_id"] == emp]
        for _, sr in emp_shifts.iterrows():
            shift_date = sr['shift_date']
            if shift_date != date_unb:
                continue

            shift_start = sr['start_time']
            shift_end = sr['end_time']

            # If no specific times are given → treat as fully unavailable
            if pd.isna(start_unb) or pd.isna(end_unb):
                errors.append(
                    f"Employee {emp} scheduled on fully unavailable day {date_unb}, shift {sr['shift_id']}."
                )
                continue

            # Check overlap
            overlap = not (shift_end <= start_unb or shift_start >= end_unb)
            if overlap:
                errors.append(
                    f"Employee {emp} scheduled during unavailable time on {date_unb}, "
                    f"shift {sr['shift_id']} ({shift_start}–{shift_end} overlaps with {start_unb}–{end_unb})."
                )

    ## 7.1) Max 5 consecutive nights
    for emp, group in assignments_df.groupby("employee_id"):
        emp_nights = group[group['is_night'] == True].sort_values("absolute_day")
        days = emp_nights['absolute_day'].tolist()
        if emp in COA_7_1_exempt:
            max_consec = 7
        else:
            max_consec = 5
        for i in range(len(days)-max_consec):
            window = days[i:i+max_consec+1]
            if window[-1] - window[0] == max_consec:  # 6 or 8 consecutive days
                errors.append(f"Employee {emp} works >5 consecutive nights (days {window}).")

    ## 7.2) After ≥3 consecutive nights → 46h rest (next 2 days off)
    for emp, group in assignments_df.groupby("employee_id"):
        emp_nights = group[group["is_night"] == True].sort_values("absolute_day")
        night_days = emp_nights["absolute_day"].tolist()

        for i in range(len(night_days) - 2):
            d0, d1, d2 = night_days[i:i+3]

            # Check if these three nights are consecutive
            if d1 == d0 + 1 and d2 == d1 + 1:
                next_day = d2 + 1

                # Check if next_day exists in planning horizon
                horizon_days = assignments_df["absolute_day"].unique()
                if next_day not in horizon_days:
                    continue  # No next day, can't enforce rule

                # Check if next_day is NOT a night shift (i.e., this is end-of-block)
                has_night_next_day = any(emp_nights["absolute_day"] == next_day)
                if has_night_next_day:
                    continue  # Still part of a longer block, skip check

                # Get all shifts on d2+1 and d2+2 (if they exist)
                blocked_days = [d2 + 1, d2 + 2]
                next_shifts = group[group["absolute_day"].isin(blocked_days)]

                if len(next_shifts) > 0:
                    errors.append(
                        f"Employee {emp} has shifts {next_shifts['shift_id'].tolist()} "
                        f"within 46h rest after 3 consecutive nights ending on day {d2} ({[d0, d1, d2]})."
                    )

    ## 7.3) Max 35 nights per 13 weeks
    for emp, group in assignments_df.groupby("employee_id"):
        emp_nights = group[group['is_night'] == True]
        for start_week in range(shifts['week'].min(), shifts['week'].max()-12):
            window = emp_nights[(emp_nights['week'] >= start_week) & (emp_nights['week'] < start_week+13)]
            if len(window) > 35:
                errors.append(f"Employee {emp} has {len(window)} nights in weeks {start_week}-{start_week+12} (> 35).")

    ## 7.4) Age > 55: no night shifts
    for emp, group in assignments_df.groupby("employee_id"):
        if emp in CAO_7_4_exempt:
            continue
        
        leeftijd = int(workers.loc[workers['medewerker_id'] == emp, 'leeftijd'].iloc[0])
        if leeftijd > 55:
            if group['is_night'].any():
                errors.append(f"Employee {emp} (age {leeftijd}) assigned to night shifts: {group[group['is_night']==True]['shift_id'].tolist()}")

    ## 8) Qualification match
    for _, row in assignments_df.iterrows():
        sid = row['shift_id']
        required_quals = shifts_lookup.loc[sid, 'qualification']
        if not isinstance(required_quals, list):
            required_quals = [required_quals]
        emp_quals = workers.loc[workers['medewerker_id'] == row['employee_id'], 'deskundigheid'].iloc[0]
        if not any(q in emp_quals for q in required_quals):
            errors.append(f"Employee {row['employee_id']} lacks qualification {required_quals} for shift {sid}.")

        ## 9) Employee-specific hard constraints
    for _, row in assignments_df.iterrows():
        emp = row['employee_id']
        sid = row['shift_id']
        shift = shifts_lookup.loc[sid]
        day_of_week = int(shift['day_of_week'])
        is_night = shift['is_night']

        # 9.1) Hennie van den Brink (2097): Alleen nachtdiensten
        if emp == '2097' and not is_night:
            errors.append(
                f"Employee 2097 (Hennie van den Brink) assigned to non-night shift {sid} (day {day_of_week})."
            )

        # 9.2) Romy van Dijkhuizen (2653): Alleen weekenden (day_of_week 5=Sat, 6=Sun)
        if emp == '2653' and day_of_week not in [5, 6]:
            errors.append(
                f"Employee 2653 (Romy van Dijkhuizen) assigned to weekday shift {sid} (day {day_of_week})."
            )

        # 9.3) Jasmijn van den Berg (3888): Alleen vrijdagavond en weekenden
        if emp ==' 3888':
            allowed = (day_of_week == 4 and is_night) or day_of_week in [5, 6]
            if not allowed:
                errors.append(
                    f"Employee 3888 (Jasmijn van den Berg) assigned to disallowed shift {sid} "
                    f"(day {day_of_week}, night={is_night})."
                )
        
        # 9.4) Maike van Kasbergen (601011): Maandag, dinsdag, woensdag geen diensten
        if emp == '601011' and day_of_week in [0, 1, 2]:
            errors.append(
                f"Employee 601011 (Maike van Kasbergen) assigned to shift {sid} on day that is not allowed: {day_of_week}.")
        
        # 9.5) Marlies Schreurs (603722): Max 2 days in a row, and after a block of shifts, always 2 days off
        if emp == '603722':
            # Get all days Marlies is scheduled
            marlies_days = assignments_df[assignments_df['employee_id'] == emp]['absolute_day'].sort_values().tolist()
            # Find blocks of consecutive days
            blocks = []
            block = []
            for d in marlies_days:
                if not block or d == block[-1] + 1:
                    block.append(d)
                else:
                    blocks.append(block)
                    block = [d]
            if block:
                blocks.append(block)
            # Check max 2 days in a row
            for b in blocks:
                if len(b) > 2:
                    errors.append(
                    f"Employee 603722 (Marlies Schreurs) works more than 2 days in a row: days {b}."
                    )
            # After each block, check for 2 days off
            for b in blocks:
                last_day = b[-1]
                next_two_days = [last_day + 1, last_day + 2]
                next_shifts = assignments_df[
                    (assignments_df['employee_id'] == emp) &
                    (assignments_df['absolute_day'].isin(next_two_days))
                ]
                if len(next_shifts) > 0:
                    errors.append(
                    f"Employee 603722 (Marlies Schreurs) does not have 2 days off after working days {b}."
                    )
        
        # 9.6) Teuna de Groot (602859): 7 nights in a row, then 7 days off
        if emp == '602859':
            nights = assignments_df[
                (assignments_df['employee_id'] == emp) &
                (assignments_df['is_night'] == True)
            ]['absolute_day'].sort_values().tolist()

            # Find consecutive night blocks
            block = []
            blocks = []
            for d in nights:
                if not block or d == block[-1] + 1:
                    block.append(d)
                else:
                    blocks.append(block)
                    block = [d]
            if block:
                blocks.append(block)

            for i, b in enumerate(blocks):
                # Allow first and last blocks to be shorter than 7
                if i != 0 and i != len(blocks)-1 and len(b) != 7:
                    errors.append(
                        f"Employee 602859 (Teuna de Groot) has night block of length {len(b)} (expected 7): days {b}."
                    )

                # Check following 7 days are OFF, but skip if block ends at horizon
                following = [d for d in range(b[-1]+1, b[-1]+8)]
                # only keep days that exist in assignments
                following = [d for d in following if d in assignments_df['absolute_day'].values]
                next_shifts = assignments_df[
                    (assignments_df['employee_id'] == emp) &
                    (assignments_df['absolute_day'].isin(following))
                ]
                if len(next_shifts) > 0:
                    errors.append(
                        f"Employee 602859 (Teuna de Groot) did not get 7 days off after block {b}."
                    )
            
        # 9.7) Maria Veldhuis-Rivera (627311): Max 3 shifts per week
        if emp == '627311':
            emp_group = assignments_df[assignments_df['employee_id'] == emp]
            for week, week_group in emp_group.groupby("week"):
                num_shifts = len(week_group)
                if num_shifts > 3:
                    errors.append(
                        f"Employee 627311 (Maria Veldhuis-Rivera) assigned to {num_shifts} shifts in week {week} (> 3)."
                    )
        
        # 9.8) Carolien van de Koppel (602056): Max 3 shifts per week and only late shifts (A or N)
        if emp == '602056':
            emp_group = assignments_df[assignments_df['employee_id'] == emp]
            for week, week_group in emp_group.groupby("week"):
                num_shifts = len(week_group)
                if num_shifts > 3:
                    errors.append(
                        f"Employee 602056 (Carolien van de Koppel) assigned to {num_shifts} shifts in week {week} (> 3)."
                    )
                for _, shift_row in week_group.iterrows():
                    shift_name = shifts_lookup.loc[shift_row['shift_id'], 'shift_name']
                    if 'A' not in shift_name and not shift_row['is_night']:
                        errors.append(
                            f"Employee 602056 (Carolien van de Koppel) assigned to non-late shift {shift_row['shift_id']} ({shift_name})."
                        )

    
    if errors:
        print(f"=== VALIDATION FAILED: {len(errors)} issue(s) ===")
        for e in errors:
            print("❌", e)
    else:
        print("✅ All constraints satisfied!")

In [95]:
validate_auto_rooster(result["assignments_df"], 
                                                     preprocessed_data['shifts'],
                                                        preprocessed_data['workers'],
                                                        preprocessed_data['dur_min'],
                                                        preprocessed_data['onb'],
                                                        preprocessed_data['night_shifts'],
                                                        preprocessed_data['night_shifts_by_week'])

✅ All constraints satisfied!


# Auto-rooster that uses old schedule as input

## Preprocessing function

In [167]:

def preprocess_data(df_shifts: pd.DataFrame, df_werknemers: pd.DataFrame, df_onb: pd.DataFrame, prev_assignments: pd.DataFrame, num_weeks: int = 4):
    """
    Prepares shifts and workers dataframes for the OR-Tools scheduling model.
    Returns processed shifts, workers, and helper structures.
    """

    # --- Shifts ---
    # Create clean shifts dataframe
    # Delete all rows where shifts = KOK or FM
    df_shifts = df_shifts[~df_shifts['Shifts'].isin(['KOK', 'FM'])]

    # Convert to long format: each row = one shift on one day
    days = ['Maandag', 'Dinsdag', 'Woensdag', 'Donderdag', 'Vrijdag', 'Zaterdag', 'Zondag']
    day_map = {day: i for i, day in enumerate(days)}  # Map day names to day indices (0–6)

    shift_requirements = []

    for idx, row in df_shifts.iterrows():
        for day in days:
            if row[day] == 1:  # If this shift needs to be filled on this day
                shift_requirements.append({
                    "shift_name": row['Shifts'],
                    "day_of_week": day_map[day],  # 0=Monday, etc.
                    "start_time": row['Begintijd'],
                    "end_time": row['Eindtijd'],
                    "qualification": row['Deskundigheden'],
                    "duration": row['Duur']
                })

    all_shift_instances = []

    # Start date is one day after last assignment date in prev_assignments
    # Ensure shift_date is datetime, not string
    if prev_assignments['shift_date'].dtype == object:
        prev_assignments['shift_date'] = pd.to_datetime(prev_assignments['shift_date'])
    
    #Make employee_id a string, delete everything after the first . (dot)
    prev_assignments['employee_id'] = prev_assignments['employee_id'].astype(str).apply(lambda x: x.split('.')[0] if '.' in x else x)    
    
    start_date = prev_assignments['shift_date'].max() + dt.timedelta(days=1)
    
    for week in range(num_weeks):
        for row in shift_requirements:
            # Compute shift_date based on start_date
            shift_date = start_date + dt.timedelta(days=week*7 + row["day_of_week"])
            all_shift_instances.append({
                **row,
                "week": week + 1,
                "absolute_day": week * 7 + row["day_of_week"],  # 0-based absolute day
                "shift_date": shift_date
            })


    shifts = pd.DataFrame(all_shift_instances)
    shifts['shift_id'] = shifts.index.astype(int)    
    
    # Convert durations (hours → minutes, integers for CP-SAT)
    shifts['duration_min'] = (shifts['duration'].astype(float) * 60).round().astype(int)

    # Ensure int types
    shifts['week'] = shifts['week'].astype(int)
    shifts['day_of_week'] = shifts['day_of_week'].astype(int)

    # Day key for grouping
    shifts['day_key'] = list(zip(shifts['week'], shifts['day_of_week']))

    # add is_night column based on shift_name (e.g., 'N' is night)
    shifts['is_night'] = shifts['shift_name'].apply(lambda x: 1 if x == 'N' else 0)
    shifts['is_night'] = shifts['is_night'].astype(bool)
    shifts['shift_id'] = shifts.index.astype(int)
    
    shifts['qualification'] = shifts['qualification'].apply(
        lambda q: ast.literal_eval(q) if isinstance(q, str) else q)
    
    shifts = shifts[['shift_id', 'shift_name', 'shift_date', 'week', 'day_of_week', 'absolute_day', 'start_time', 'end_time', 'duration_min', 'qualification', 'is_night', 'day_key']]
    
    # --- Workers ---
    workers = df_werknemers.copy().reset_index(drop=True)
    workers['medewerker_id'] = workers['medewerker_id'].astype(str)
    
    # DELETE REIJNDERS SINCE THIS IS AN ANOMALY (CHANGE LATER)
    workers = workers[workers['medewerker_naam'] != 'Reijnders']
    workers = workers.reset_index(drop=True)
    
    # Change werknemers_id to delete every 0 at the start and everything and including the first -
    workers['medewerker_id'] = workers['medewerker_id'].apply(lambda x: x.lstrip('0').split('-')[0])
    
    # delete all rows where wensen = niet plannen
    workers = workers[workers['wensen'] != 'niet plannen']
    workers = workers.reset_index(drop=True)
    
    # Convert contract geldig van, contract tm, and geboortedatum to datetime
    workers['contract_vanaf'] = pd.to_datetime(workers['contact vanaf'], format='%d/%m/%Y')
    workers['contract_tm'] = pd.to_datetime(workers['contract tm'], format='%d/%m/%Y')
    #If contract_tm is NaT, set to 31-12-2099
    workers['contract_tm'] = workers['contract_tm'].fillna(pd.Timestamp('2099-12-31'))
    
    # Drop unnecessary columns
    workers = workers.drop(columns=['contact vanaf', 'contract tm'])
    
    #Convert deskundigheid to list of ints
    lsts = []
    for i in range(len(workers)):
        lst = []
        if workers['deskundigheid'][i] <10:
            lst = [int(workers['deskundigheid'][i])]
        else:
            lst = [int(x) for x in str(int(workers['deskundigheid'][i]))]
        
        lsts.append(lst)
        
    workers['deskundigheid'] = lsts

    workers['deskundigheid'] = workers['deskundigheid'].apply(
        lambda q: ast.literal_eval(q) if isinstance(q, str) else q
    )
    # Delete all employees with deskundigheid = 5 or 6
    workers = workers[~workers['deskundigheid'].apply(lambda x: isinstance(x, (list, tuple)) and (5 in x or 6 in x or 7 in x))]
    
    # If contract soort = oproep, set contracturen to 0
    workers.loc[workers['contract soort'] == 'oproep', 'contracturen'] = 0
    
    # Normalize contract details
    workers['max_days_per_week'] = workers['max_werkdgn_pw'].fillna(0).astype(int)
    
    # Safely convert contracturen to float, fill NaN with 0 before converting to int
    workers['contract_hours'] = workers['contracturen'].fillna(0).astype(float)
    
    # if contract_hours is 0, fill with 9 * max_days_per_week
    workers.loc[workers['contract_hours'] == 0, 'contract_hours'] = workers['max_days_per_week'] * 9
    
    workers['contract_minutes'] = (workers['contract_hours'] * 60).round().astype(int) 
    # Convert geboortedatum to leeftijd based on current day (round down if birthday not yet occurred this year)
    current_date = dt.datetime.now().date()

    workers['leeftijd'] = workers['geboortedatum'].apply(
        lambda x: current_date.year - x.year - ((current_date.month, current_date.day) < (x.month, x.day))
    )
    
    # Create IDs
    emp_ids = workers['medewerker_id'].tolist()
    #emp_ids = [str(e) for e in emp_ids]
    emp_index = {emp: i for i, emp in enumerate(emp_ids)}

    # --- onhbeschikbaarheid ---
    # Convert 'Datum beschikbaarheid' to datetime
    df_onb['Datum'] = pd.to_datetime(df_onb['Datum beschikbaarheid'], format='%d-%m-%Y').dt.date
    df_onb['Beschikbaarheid'] = df_onb['Beschikbaarheid'].fillna('Onbekend')
    df_onb['Beschikbaarheid_tijd_vanaf'] = pd.to_datetime(df_onb['Beschikbaarheid tijd vanaf'], format='%H:%M', errors='coerce').dt.time
    df_onb['Beschikbaarheid_tijd_tm'] = pd.to_datetime(df_onb['Beschikbaarheid tijd t/m'], format='%H:%M', errors='coerce').dt.time
    df_onb = df_onb[['Medewerker id', 'Datum', 'Beschikbaarheid', 'Beschikbaarheid_tijd_vanaf', 'Beschikbaarheid_tijd_tm']]
    df_onb['Medewerker id'] = df_onb['Medewerker id'].astype(str)

    # --- Helper dictionaries ---
    dur_min = shifts.set_index('shift_id')['duration_min'].to_dict()

    # Grouping by week/day
    shifts_by_week = {}
    #shifts_by_day = {}
    for _, r in shifts.iterrows():
        s = int(r['shift_id'])
        w = int(r['week'])
        day_k = r['day_key']
        shifts_by_week.setdefault(w, []).append(s)
        #shifts_by_day.setdefault(day_k, []).append(s)

    shifts_by_day = shifts.groupby('absolute_day')['shift_id'].apply(list).to_dict()
    
    weeks = sorted(shifts['week'].unique().tolist())
    
    night_shifts = shifts.loc[shifts['is_night'], 'shift_id'].tolist()
    night_shifts_by_week = {
        w: shifts.loc[(shifts['week'] == w) & (shifts['is_night']), 'shift_id'].tolist() for w in weeks}

    return {
        "shifts": shifts,
        "workers": workers,
        "onb": df_onb,
        "emp_ids": emp_ids,
        "emp_index": emp_index,
        "dur_min": dur_min,
        "shifts_by_week": shifts_by_week,
        "shifts_by_day": shifts_by_day,
        "weeks": weeks,
        "night_shifts": night_shifts,
        "night_shifts_by_week": night_shifts_by_week,
        "prev_assignments": prev_assignments
    }

In [168]:
info_shifts = {
    "Shifts": ['D1', 'D2', 'D3', 'D4', 'FM', 'GD', 'KOK', 'A1', 'A2', 'A3', 'GA', 'N'],
    "Begintijd": ['07:00', '07:30', '07:30', '08:45', '08:00', '08:00', '15:00', '15:00', '15:30', '18:30', '16:00', '22:45'],
    "Eindtijd": ['15:00', '15:00', '14:00', '12:45', '12:30', '14:00', '19:30', '23:00', '22:30', '22:30', '22:00', '07:00'],
    "Duur": [8, 7.5, 6.5, 4, 4.5, 6, 4.5, 8, 7, 4, 6, 8.25],
    "Maandag": [1,1,1,1,1,1,1,1,1,1,1,1],
    "Dinsdag": [1,1,1,1,1,1,1,1,1,1,1,1],
    "Woensdag": [1,1,1,1,1,1,1,1,1,1,1,1],
    "Donderdag": [1,1,1,0,1,1,1,1,1,1,1,1],
    "Vrijdag": [1,1,1,1,0,1,1,1,1,1,1,1],
    "Zaterdag": [1,1,1,0,0,1,1,1,1,1,1,1],
    "Zondag": [1,1,1,1,0,1,1,1,1,0,1,1],
    "Deskundigheden": [[1,2], [2], [3], [3], [6], [4], [5], [2], [3], [3], [4], [2]]
}

for i in range(len(info_shifts['Shifts'])):
    info_shifts['Begintijd'][i] = dt.datetime.strptime(info_shifts['Begintijd'][i], '%H:%M').time()
    info_shifts['Eindtijd'][i] = dt.datetime.strptime(info_shifts['Eindtijd'][i], '%H:%M').time()
    
df_shifts = pd.DataFrame(info_shifts)

df_werknemers = pd.read_excel('data\medewerkers_compleet.xlsx')

df_onb = pd.read_csv('data/Onbeschikbaarheid_hoevelaken.csv', sep=';')
df_onb['Datum'] = pd.to_datetime(df_onb['Datum beschikbaarheid'], format='%d-%m-%Y').dt.date
df_onb['Beschikbaarheid'] = df_onb['Beschikbaarheid'].fillna('Onbekend')

last_schedule = pd.read_csv('output/shift_assignments_01_09_2025.csv')

preprocessed_data = preprocess_data(df_shifts, df_werknemers, df_onb, last_schedule)
preprocessed_data["prev_assignments"]

C:\Users\20192507\AppData\Local\Temp\ipykernel_18176\1916725609.py:152: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_onb['Medewerker id'] = df_onb['Medewerker id'].astype(str)


,shift_id,shift_name,start_time,end_time,shift_date,is_night,week,day_of_week,absolute_day,duration_min,employee_id,employee_name,qualification,deskundigheid
0,0,D1,07:00:00,15:00:00,2025-09-01,False,1,0,0,480,599363,Arja van Pelt-Ymker,"[1, 2]",[1]
1,1,D1,07:00:00,15:00:00,2025-09-02,False,1,1,1,480,599363,Arja van Pelt-Ymker,"[1, 2]",[1]
2,2,D1,07:00:00,15:00:00,2025-09-03,False,1,2,2,480,599354,Ali van Soeren,"[1, 2]",[1]
3,3,D1,07:00:00,15:00:00,2025-09-04,False,1,3,3,480,599363,Arja van Pelt-Ymker,"[1, 2]",[1]
4,4,D1,07:00:00,15:00:00,2025-09-05,False,1,4,4,480,599354,Ali van Soeren,"[1, 2]",[1]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,263,N,22:45:00,07:00:00,2025-09-24,True,4,2,23,495,602859,Teuna de Groot,[2],[2]
264,264,N,22:45:00,07:00:00,2025-09-25,True,4,3,24,495,602859,Teuna de Groot,[2],[2]
265,265,N,22:45:00,07:00:00,2025-09-26,True,4,4,25,495,2097,Hennie van den Brink,[2],[2]
266,266,N,22:45:00,07:00:00,2025-09-27,True,4,5,26,495,599862,Irma van Diermen,[2],[2]


## Auto-rooster function

In [180]:

def auto_rooster(data, time_limit_s=60):
    import pandas as pd
    from ortools.sat.python import cp_model
    


    shifts = data['shifts'].copy()
    workers = data['workers'].copy()
    onb = data['onb'].copy()
    emp_ids = data['emp_ids']
    dur_min = data['dur_min']
    shifts_by_week = data['shifts_by_week']
    weeks = data['weeks']
    night_shifts = data['night_shifts']
    night_shifts_by_week = data['night_shifts_by_week']
    prev_assignments = data['prev_assignments']
    

    if prev_assignments is None:
        prev_assignments = pd.DataFrame(columns=[
            'shift_id', 'shift_date', 'employee_id', 'is_night', 'absolute_day', 'week'
        ])
    
    print("=== AUTO ROOSTER START ===")
    print(f"Planning horizon: weeks {weeks[0]}–{weeks[-1]} ({len(weeks)} weeks)")
    print(f"start_date: {shifts['shift_date'].min().date()}, end_date: {shifts['shift_date'].max().date()}")
    print(f"Shifts: {len(shifts)}, Workers: {len(workers)}")

    # make sure shift_date is datetime (date or Timestamp)
    shifts['shift_date'] = pd.to_datetime(shifts['shift_date']).dt.normalize()  # midnight timestamps
    prev_assignments['shift_date'] = pd.to_datetime(prev_assignments['shift_date']).dt.normalize()

    # helper lists and maps using dates
    dates_list = sorted(shifts['shift_date'].dt.date.unique().tolist())  # list of date objects
    num_weeks = len(weeks)

    # shifts_by_date: date -> [shift_id, ...]
    shifts_by_date = {}
    for _, r in shifts.iterrows():
        s = int(r['shift_id'])
        d = pd.to_datetime(r['shift_date']).date()
        shifts_by_date.setdefault(d, []).append(s)

    # night_shifts_by_date: date -> [night_shift_id,...]
    night_shifts_by_date = {}
    for _, r in shifts[shifts['is_night']].iterrows():
        s = int(r['shift_id'])
        d = pd.to_datetime(r['shift_date']).date()
        night_shifts_by_date.setdefault(d, []).append(s)

    # helper to get previous consecutive block using dates (returns list of dates)
    def get_last_consecutive_block_dates(prev_assignments_df, emp, is_night=False):
        df = prev_assignments_df[prev_assignments_df['employee_id'] == emp].copy()
        if is_night:
            df = df[df['is_night'] == True]
        if df.empty:
            return []
        # normalized dates sorted ascending
        days = sorted(pd.to_datetime(df['shift_date']).dt.date.unique().tolist())
        # walk from end backwards to collect consecutive tail
        block = []
        for d in reversed(days):
            if not block:
                block.append(d)
            else:
                if (pd.to_datetime(block[-1]) - pd.to_datetime(d)).days == 1:
                    block.append(d)
                else:
                    break
        return list(reversed(block))

    model = cp_model.CpModel()

    # Decision variables: x[(shift_id, emp)]
    x = {(s, emp): model.NewBoolVar(f"x_s{s}_e{emp}")
         for s in shifts['shift_id'] for emp in emp_ids}

    # uncovered
    u = {s: model.NewBoolVar(f"uncovered_s{s}") for s in shifts['shift_id']}

    # n_emp_date: 1 iff employee emp works a night on date d (date object)
    n_emp_date = {}
    for emp in emp_ids:
        for d in dates_list:
            b = model.NewBoolVar(f"n_emp{emp}_d{d.isoformat()}")
            n_emp_date[(emp, d)] = b
            night_ids = night_shifts_by_date.get(d, [])
            if night_ids:
                # if b then sum >=1, else sum == 0
                model.Add(sum(x[(s, emp)] for s in night_ids) >= 1).OnlyEnforceIf(b)
                model.Add(sum(x[(s, emp)] for s in night_ids) == 0).OnlyEnforceIf(b.Not())
            else:
                model.Add(b == 0)

    ### Constraints ###

    # 1) Coverage
    for s in shifts['shift_id']:
        model.Add(sum(x[(s, emp)] for emp in emp_ids) + u[s] == 1)

    # 2) At most one shift per employee per calendar day (use shifts_by_date)
    for emp in emp_ids:
        for d, s_list in shifts_by_date.items():
            if s_list:
                model.AddAtMostOne(x[(s, emp)] for s in s_list)

    # 3) After night shift, no day/evening next day (but night allowed next day)
    # For each emp and each date d: if emp works any night on d then they cannot work non-night shifts on d+1
    for emp in emp_ids:
        for d in dates_list:
            next_d = (pd.to_datetime(d) + pd.Timedelta(days=1)).date()
            if next_d not in shifts_by_date:
                continue
            night_ids_today = night_shifts_by_date.get(d, [])
            next_day_non_night_ids = [s for s in shifts_by_date.get(next_d, []) if not bool(shifts.loc[shifts['shift_id'] == s, 'is_night'].iloc[0])]
            if not night_ids_today or not next_day_non_night_ids:
                continue
            # sum(x_night_today) + sum(x_non_night_nextday) <= len(night_ids_today)
            # if any night worked today -> none of next_day_non_night_ids can be worked.
            model.Add(
                sum(x[(s, emp)] for s in night_ids_today) +
                sum(x[(s, emp)] for s in next_day_non_night_ids)
                <= len(night_ids_today)
            )

    # 4) Max work days per week (kept weekly using shifts_by_week)
    for emp in emp_ids:
        max_days = int(workers.loc[workers['medewerker_id'] == emp, 'max_days_per_week'].iloc[0])
        for w in weeks:
            s_list = shifts_by_week.get(w, [])
            if s_list:
                model.Add(sum(x[(s, emp)] for s in s_list) <= max_days)

    # 5) Contract hours averaged across horizon
    for emp in emp_ids:
        cap_minutes = int(workers.loc[workers['medewerker_id'] == emp, 'contract_minutes'].iloc[0])
        total_shifts = list(shifts['shift_id'].tolist())
        model.Add(sum(dur_min[s] * x[(s, emp)] for s in total_shifts) <= cap_minutes * num_weeks)

    # 6) Respect unavailable times (date-aware)
    # assume onb['Datum'] is a date or datetime; times in 'Beschikbaarheid_tijd_vanaf' and '_tm' are time-like
    for _, r in onb.iterrows():
        emp = r['Medewerker id']
        if emp not in emp_ids:
            continue
        date_unb = pd.to_datetime(r['Datum']).date()
        besch = str(r['Beschikbaarheid']).lower()
        if besch == 'beschikbaar':
            continue
        start_unb = r.get('Beschikbaarheid_tijd_vanaf', None)
        end_unb = r.get('Beschikbaarheid_tijd_tm', None)
        # if times are strings, try to parse to time
        if isinstance(start_unb, str):
            start_unb = pd.to_datetime(start_unb).time()
        if isinstance(end_unb, str):
            end_unb = pd.to_datetime(end_unb).time()
        shifts_that_day = shifts_by_date.get(date_unb, [])
        for s in shifts_that_day:
            shift_start = shifts.loc[shifts['shift_id'] == s, 'start_time'].iloc[0]
            shift_end = shifts.loc[shifts['shift_id'] == s, 'end_time'].iloc[0]
            # overlap check
            if start_unb is not None and end_unb is not None:
                overlap = not (shift_end <= start_unb or shift_start >= end_unb)
            else:
                # if no times provided, treat as fully unavailable that day
                overlap = True
            if overlap:
                model.Add(x[(s, emp)] == 0)

    # 7.1) Max consecutive nights (consider prev_assignments)
    for emp in emp_ids:
        max_consec = 7 if emp in {str(602859)} else 5  # your CAO exemption set uses strings in your code earlier
        # previous night dates as date objects
        prev_night_dates = get_last_consecutive_block_dates(prev_assignments, emp, is_night=True)
        # current nights dates (unique sorted)
        curr_night_dates = sorted(night_shifts_by_date.keys())
        combined = sorted(set(prev_night_dates + curr_night_dates), key=lambda d: pd.to_datetime(d))
        # slide window of size max_consec + 1 over combined dates and detect strictly consecutive windows
        for i in range(len(combined) - max_consec):
            window = combined[i:i + max_consec + 1]
            # check if strictly consecutive by date difference
            if (pd.to_datetime(window[-1]) - pd.to_datetime(window[0])).days == max_consec:
                # identify which dates in window are in *current horizon*
                window_curr_dates = [d for d in window if d in curr_night_dates]
                if not window_curr_dates:
                    continue
                # find all night shift ids in the current horizon that fall on those dates
                night_shift_ids_in_window = [s for d in window_curr_dates for s in night_shifts_by_date.get(d, [])]
                # enforce at most max_consec nights in that window
                model.Add(sum(x[(s, emp)] for s in night_shift_ids_in_window) <= max_consec)

    # 7.2) After >=3 consecutive nights => 46h rest (2 calendar days)
    # Build boolean n_emp_date already created. Use OnlyEnforceIf with cond_lits.
    for emp in emp_ids:
        # handle prev_assignments tail: if the last prev block >= 3 and ends on day Dprev,
        # then block Dprev+1 and Dprev+2 (if in current horizon)
        prev_nights = get_last_consecutive_block_dates(prev_assignments, emp, is_night=True)
        if len(prev_nights) >= 3:
            dprev = prev_nights[-1]
            for k in [1, 2]:
                block_d = (pd.to_datetime(dprev) + pd.Timedelta(days=k)).date()
                if block_d in dates_list:
                    # block all shifts that day for this employee
                    for s in shifts_by_date.get(block_d, []):
                        model.Add(x[(s, emp)] == 0)

        # now enforce within horizon using OnlyEnforceIf with n_emp_date variables
        for idx in range(2, len(dates_list)):
            d2 = dates_list[idx]
            d1 = dates_list[idx - 1]
            d0 = dates_list[idx - 2]
            # need d2+1 to exist to form condition "not n(d2+1)"
            d2p1 = (pd.to_datetime(d2) + pd.Timedelta(days=1)).date()
            if d2p1 not in n_emp_date and d2p1 not in dates_list:
                continue
            # cond literals: n(d0) & n(d1) & n(d2) & not n(d2+1)
            cond = [
                n_emp_date[(emp, d0)],
                n_emp_date[(emp, d1)],
                n_emp_date[(emp, d2)]
            ]
            # n(d2+1) might not exist if outside horizon; treat absent as 0; but we only enforce if d2+1 is in horizon
            if d2p1 in dates_list:
                cond.append(n_emp_date[(emp, d2p1)].Not())
            else:
                # if next day not in horizon we treat condition only requiring n(d0)&n(d1)&n(d2)
                # but we won't be able to block shifts in missing dates, so skip
                continue

            # blocked days: d2+1 and d2+2 if they exist in horizon
            blocked_days = []
            for k in [1, 2]:
                bd = (pd.to_datetime(d2) + pd.Timedelta(days=k)).date()
                if bd in dates_list:
                    blocked_days.append(bd)
            # collect blocked shift ids
            blocked_shift_ids = [s for bd in blocked_days for s in shifts_by_date.get(bd, [])]
            for bs in blocked_shift_ids:
                model.Add(x[(bs, emp)] == 0).OnlyEnforceIf(cond)

    # 7.3) Max 35 nights per 13 weeks (count prev nights in sliding windows)
    for emp in emp_ids:
        # get previous night shift dates (not necessarily consecutive)
        prev_nights_all = prev_assignments[(prev_assignments['employee_id'] == emp) & (prev_assignments['is_night'] == True)].copy()
        prev_night_shift_ids = prev_nights_all['shift_id'].tolist()

        # for each 13-week window based on your weeks list
        min_week = min(weeks)
        max_week = max(weeks)
        for start_week in range(min_week, max_week - 12 + 1):
            window_weeks = list(range(start_week, start_week + 13))
            window_shifts = [s for w in window_weeks for s in night_shifts_by_week.get(w, [])]
            if not window_shifts:
                continue
            # count how many previous-night shifts fall in these weeks
            prev_count = 0
            for s_prev in prev_night_shift_ids:
                # need mapping shift_id -> week for prev shift (assumed same calendar mapping as shifts)
                try:
                    w_prev = int(prev_assignments.loc[prev_assignments['shift_id'] == s_prev, 'week'].iloc[0])
                    if w_prev in window_weeks:
                        prev_count += 1
                except Exception:
                    # if prev doesn't contain week, skip (or you could compute from date)
                    pass
            model.Add(sum(x[(s, emp)] for s in window_shifts) + prev_count <= 35)

    # 7.4) Age > 55: no night shifts (respect exemptions)
    CAO_7_4_exempt = {str(2097), str(602859)}
    for emp in emp_ids:
        if emp in CAO_7_4_exempt:
            continue
        leeftijd = int(workers.loc[workers['medewerker_id'] == emp, 'leeftijd'].iloc[0])
        if leeftijd > 55:
            for s in night_shifts:
                model.Add(x[(s, emp)] == 0)

    # 8) Qualification
    for _, shift_row in shifts.iterrows():
        sid = int(shift_row['shift_id'])
        required_quals = shift_row['qualification']
        if not isinstance(required_quals, list):
            required_quals = [required_quals]
        for emp in emp_ids:
            emp_quals = workers.loc[workers['medewerker_id'] == emp, 'deskundigheid'].iloc[0]
            if not any(q in emp_quals for q in required_quals):
                model.Add(x[(sid, emp)] == 0)

    # ---------- Wensen (date-aware) ----------

    # 9.1) Hennie (2097) only nights
    emp_vdBrink = '2097'
    if emp_vdBrink in emp_ids:
        for s in shifts['shift_id']:
            if s not in night_shifts:
                model.Add(x[(s, emp_vdBrink)] == 0)

    # 9.2) Romy (2653) only weekends -> use day_of_week from shift row
    emp_vDijkhuizen = '2653'
    if emp_vDijkhuizen in emp_ids:
        for _, r in shifts.iterrows():
            s = int(r['shift_id'])
            dow = int(r['day_of_week'])
            if dow not in [5, 6]:
                model.Add(x[(s, emp_vDijkhuizen)] == 0)

    # 9.3) Jasmijn (3888): Friday evening (shift_name has 'A') or weekends
    emp_vdBerg = '3888'
    if emp_vdBerg in emp_ids:
        for _, r in shifts.iterrows():
            s = int(r['shift_id'])
            dow = int(r['day_of_week'])
            shift_name = r['shift_name']
            if not ((dow == 4 and 'A' in shift_name) or dow in [5, 6]):
                model.Add(x[(s, emp_vdBerg)] == 0)

    # 9.4) Maike (601011): No Mon/Tue/Wed
    emp_vKasbergen = '601011'
    if emp_vKasbergen in emp_ids:
        for _, r in shifts.iterrows():
            s = int(r['shift_id'])
            dow = int(r['day_of_week'])
            if dow in [0, 1, 2]:
                model.Add(x[(s, emp_vKasbergen)] == 0)

    # 9.5) Marlies (603722): max 2 days in row and 2 days off after block, with prev_assignments considered
    emp_Schreurs = '603722'
    if emp_Schreurs in emp_ids:
        # day-level bools keyed by calendar date
        work_day = {}
        for d in dates_list:
            w = model.NewBoolVar(f"work_{emp_Schreurs}_{d.isoformat()}")
            work_day[d] = w
            shifts_today = shifts_by_date.get(d, [])
            if shifts_today:
                model.Add(sum(x[(s, emp_Schreurs)] for s in shifts_today) >= 1).OnlyEnforceIf(w)
                model.Add(sum(x[(s, emp_Schreurs)] for s in shifts_today) == 0).OnlyEnforceIf(w.Not())
            else:
                model.Add(w == 0)

        # incorporate previous tail block length (dates)
        prev_block = get_last_consecutive_block_dates(prev_assignments, emp_Schreurs, is_night=False)
        prev_len = len(prev_block)

        # Max 2 days in a row: for every triple d,d+1,d+2 -> sum <= 2
        for i in range(len(dates_list) - 2):
            d = dates_list[i]
            d1 = dates_list[i + 1]
            d2 = dates_list[i + 2]
            # If prev_len > 0 and i == 0, then we must account previous consecutive tail that may combine with d,d1,d2
            # Use linearization: offset = prev_len but clamp to 2 because prev_len could be >2 which is already infeasible
            offset = min(prev_len, 2) if i == 0 else 0
            # model: work(d)+work(d1)+work(d2) + offset <= 2
            model.Add(work_day[d] + work_day[d1] + work_day[d2] + offset <= 2)

        # 2 days off after block end: handle previous block that ended just before horizon
        if prev_len >= 1:
            # if prev block length >=1 and ended right before start date,
            # we must ensure the first (2 - (prev_len-?)) days are off depending on whether prev block ended >=2... simpler: if prev_len>=2 then first day must be off; if prev_len>=1 then enforce first two days off if block ended two days before? 
            # Simpler robust rule: if prev_len >= 2 then the first day must be off; if prev_len >=3 then first two days must be off.
            if prev_len >= 3:
                for d in dates_list[:2]:
                    model.Add(work_day[d] == 0)
            elif prev_len == 2:
                # require first day off
                model.Add(work_day[dates_list[0]] == 0)
            # if prev_len == 1, we don't force offs (block hasn't reached 2)
        # Also enforce "if works today and NOT tomorrow -> day after tomorrow must be off"
        for i in range(len(dates_list) - 2):
            d = dates_list[i]
            d1 = dates_list[i + 1]
            d2 = dates_list[i + 2]
            model.Add(work_day[d2] == 0).OnlyEnforceIf([work_day[d], work_day[d1].Not()])

    # 9.6) Teuna (602859): 7-on/7-off pattern using dates, respect prev_assignments
    emp_deGroot = '602859'
    if emp_deGroot in emp_ids:
        # 1) forbid non-night shifts
        for s in shifts['shift_id']:
            if s not in night_shifts:
                model.Add(x[(s, emp_deGroot)] == 0)
        # 2) day-level night bools (already have n_emp_date)
        night_work_day = {d: n_emp_date[(emp_deGroot, d)] for d in dates_list}

        # find last consecutive night block in prev_assignments (dates)
        prev_nights = get_last_consecutive_block_dates(prev_assignments, emp_deGroot, is_night=True)
        period = 14
        # compute offset if we have prior info that constitutes a consistent phase
        prev_phase_offset = None
        if prev_nights:
            # if prev tail has at least one day, we can compute candidate offset: (last_day_index mod 14)
            # choose earliest prev_night for offset calculation
            # offset = (date_index_of_prev_first - date0_index) % 14 where date_index_of_prev_first relates to a continuous day index
            # simpler: take numeric days since reference date0
            try:
                base = pd.to_datetime(dates_list[0])
                prev_first = pd.to_datetime(prev_nights[0])
                prev_phase_offset = (prev_first - base).days % period
            except Exception:
                prev_phase_offset = None

        if prev_phase_offset is not None:
            # enforce exact pattern consistent with prev_phase_offset
            for d in dates_list:
                pos = ((pd.to_datetime(d) - pd.to_datetime(dates_list[0])).days - prev_phase_offset) % period
                if pos < 7:
                    model.Add(night_work_day[d] == 1)
                else:
                    model.Add(night_work_day[d] == 0)
        else:
            # allow solver to choose a phase
            offsets = [model.NewBoolVar(f"teuna_phase_{k}") for k in range(period)]
            model.Add(sum(offsets) == 1)
            for k, vk in enumerate(offsets):
                for d in dates_list:
                    pos = ((pd.to_datetime(d) - pd.to_datetime(dates_list[0])).days - k) % period
                    if pos < 7:
                        model.Add(night_work_day[d] == 1).OnlyEnforceIf(vk)
                    else:
                        model.Add(night_work_day[d] == 0).OnlyEnforceIf(vk)

    # 9.8) Carolien (602056): max 3 shifts / week, only A or N
    emp_vdKoppel = '602056'
    if emp_vdKoppel in emp_ids:
        for w in weeks:
            s_list = shifts_by_week.get(w, [])
            if s_list:
                model.Add(sum(x[(s, emp_vdKoppel)] for s in s_list) <= 3)
                for s in s_list:
                    shift_name = shifts.loc[shifts['shift_id'] == s, 'shift_name'].iloc[0]
                    if 'A' not in shift_name and 'N' not in shift_name:
                        model.Add(x[(s, emp_vdKoppel)] == 0)

    # Objective: minimize uncovered shifts (same as before)
    penalty_uncovered = 1
    model.Minimize(penalty_uncovered * sum(u.values()))

    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = time_limit_s
    solver.parameters.num_search_workers = 8
    status = solver.Solve(model)

    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        assignments = []
        uncovered = []
        for _, r in shifts.iterrows():
            sid = int(r['shift_id'])
            assigned = False
            for emp in emp_ids:
                if solver.Value(x[(sid, emp)]) == 1:
                    assignments.append({
                        'shift_id': sid,
                        'shift_name': r['shift_name'],
                        'start_time': r['start_time'],
                        'end_time': r['end_time'],
                        'shift_date': r['shift_date'],
                        'is_night': r['is_night'],
                        'week': int(r['week']),
                        'day_of_week': int(r['day_of_week']),
                        'absolute_day': int(r['absolute_day']),
                        'duration_min': int(r['duration_min']),
                        'employee_id': str(emp),
                        'employee_name': workers.loc[workers['medewerker_id'] == emp, 'medewerker_naam'].iloc[0],
                        'qualification': r['qualification'],
                        'deskundigheid': workers.loc[workers['medewerker_id'] == emp, 'deskundigheid'].iloc[0]
                    })
                    assigned = True
                    break
            if not assigned:
                uncovered.append(sid)
                assignments.append({
                    'shift_id': sid,
                    'shift_name': r['shift_name'],
                    'start_time': r['start_time'],
                    'end_time': r['end_time'],
                    'shift_date': r['shift_date'],
                    'is_night': r['is_night'],
                    'week': int(r['week']),
                    'day_of_week': int(r['day_of_week']),
                    'absolute_day': int(r['absolute_day']),
                    'duration_min': int(r['duration_min']),
                    'employee_id': None,
                    'employee_name': None,
                    'qualification': r['qualification'],
                    'deskundigheid': None
                })
        
        # append new assignments to prev_assignments
        all_assignments = pd.concat([prev_assignments, pd.DataFrame(assignments)], ignore_index=True)
        
        print(f"Solution found with objective value {len(uncovered)} uncovered shifts out of {len(shifts)} total shifts.")
        
        #save to CSV
        assignments_df = pd.DataFrame(assignments)
        assignments_df.to_csv(f"output/Rooster_{shifts['shift_date'].min().date().strftime('%d_%m_%Y')}.csv", index=False)
        print(f"Shift assignments saved to output/Rooster_{shifts['shift_date'].min().date().strftime('%d_%m_%Y')}.csv")
        
        return {
            "assignments_df": assignments_df,
            "all_assignments_df": all_assignments,
            "uncovered_shifts": uncovered,
            "objective_value": solver.ObjectiveValue(),
            "solver_status": solver.StatusName(status)
        }
    else:
        print("No solution found:", solver.StatusName(status))
        return None

In [181]:
result = auto_rooster(preprocessed_data, time_limit_s=120)

#first date in shifts
first_date = preprocessed_data['shifts']['shift_date'].min()
#format as dd_mm_yyyy
date_str = first_date.strftime('%d_%m_%Y')

result['assignments_df']

=== AUTO ROOSTER START ===
Planning horizon: weeks 1–4 (4 weeks)
start_date: 2025-09-29, end_date: 2025-10-26
Shifts: 268, Workers: 21
Solution found with objective value 26 uncovered shifts out of 268 total shifts.
Shift assignments saved to output/Rooster_29_09_2025.csv


,shift_id,shift_name,start_time,end_time,shift_date,is_night,week,day_of_week,absolute_day,duration_min,employee_id,employee_name,qualification,deskundigheid
0,0,D1,07:00:00,15:00:00,2025-09-29,False,1,0,0,480,599354,Ali van Soeren,"[1, 2]",[1]
1,1,D1,07:00:00,15:00:00,2025-09-30,False,1,1,1,480,599363,Arja van Pelt-Ymker,"[1, 2]",[1]
2,2,D1,07:00:00,15:00:00,2025-10-01,False,1,2,2,480,599354,Ali van Soeren,"[1, 2]",[1]
3,3,D1,07:00:00,15:00:00,2025-10-02,False,1,3,3,480,640971,Janneke Romkes-Visscher,"[1, 2]",[2]
4,4,D1,07:00:00,15:00:00,2025-10-03,False,1,4,4,480,605162,Eline van de Craats,"[1, 2]",[2]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,263,N,22:45:00,07:00:00,2025-10-22,True,4,2,23,495,602859,Teuna de Groot,[2],[2]
264,264,N,22:45:00,07:00:00,2025-10-23,True,4,3,24,495,602859,Teuna de Groot,[2],[2]
265,265,N,22:45:00,07:00:00,2025-10-24,True,4,4,25,495,640971,Janneke Romkes-Visscher,[2],[2]
266,266,N,22:45:00,07:00:00,2025-10-25,True,4,5,26,495,2097,Hennie van den Brink,[2],[2]


In [164]:
result['all_assignments_df']

,shift_id,shift_name,start_time,end_time,shift_date,is_night,week,day_of_week,absolute_day,duration_min,employee_id,employee_name,qualification,deskundigheid
0,0,D1,07:00:00,15:00:00,2025-09-01,False,1,0,0,480,599363,Arja van Pelt-Ymker,"[1, 2]",[1]
1,1,D1,07:00:00,15:00:00,2025-09-02,False,1,1,1,480,599363,Arja van Pelt-Ymker,"[1, 2]",[1]
2,2,D1,07:00:00,15:00:00,2025-09-03,False,1,2,2,480,599354,Ali van Soeren,"[1, 2]",[1]
3,3,D1,07:00:00,15:00:00,2025-09-04,False,1,3,3,480,599363,Arja van Pelt-Ymker,"[1, 2]",[1]
4,4,D1,07:00:00,15:00:00,2025-09-05,False,1,4,4,480,599354,Ali van Soeren,"[1, 2]",[1]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,263,N,22:45:00,07:00:00,2025-10-22,True,4,2,23,495,602859,Teuna de Groot,[2],[2]
532,264,N,22:45:00,07:00:00,2025-10-23,True,4,3,24,495,602859,Teuna de Groot,[2],[2]
533,265,N,22:45:00,07:00:00,2025-10-24,True,4,4,25,495,605162,Eline van de Craats,[2],[2]
534,266,N,22:45:00,07:00:00,2025-10-25,True,4,5,26,495,605162,Eline van de Craats,[2],[2]


In [182]:
import pandas as pd
from datetime import timedelta

def validate_auto_rooster(assignments_df, shifts, workers, dur_min, onb, night_shifts, night_shifts_by_week):
    errors = []

    # Drop unassigned shifts
    assignments_df = assignments_df.dropna(subset=['employee_id'])
    
    # Helper: quick lookup
    shifts_lookup = shifts.set_index('shift_id')

    COA_7_1_exempt = set(map(str, [602859]))  # Teuna
    CAO_7_4_exempt = set(map(str, [2097, 602859]))  # Hennie and Teuna

    ## 1) Coverage: each shift has <= 1 assigned employee
    for sid, group in assignments_df.groupby("shift_id"):
        if len(group) > 1:
            errors.append(f"Shift {sid} assigned to multiple employees: {group['employee_id'].tolist()}")

    ## 2) At most one shift per employee per day
    for (emp, date), group in assignments_df.groupby(["employee_id", "shift_date"]):
        if len(group) > 1:
            errors.append(f"Employee {emp} works {len(group)} shifts on {date}: {group['shift_id'].tolist()}")

    ## 3) After night shift, no day/evening shift next day
    for emp, group in assignments_df.groupby("employee_id"):
        nights = group[group["is_night"]]
        for _, ns in nights.iterrows():
            next_day = ns['shift_date'] + timedelta(days=1)
            next_day_shifts = group[group['shift_date'] == next_day]
            next_day_shifts = next_day_shifts[next_day_shifts['is_night'] == False]
            if len(next_day_shifts) > 0:
                errors.append(f"Employee {emp} has day/evening shift(s) on {next_day} after a night shift on {ns['shift_date']}.")

    ## 4) Max work days per week
    for emp, group in assignments_df.groupby("employee_id"):
        max_days = int(workers.loc[workers['medewerker_id'] == emp, 'max_days_per_week'].iloc[0])
        for week, week_group in group.groupby("week"):
            worked_days = week_group['shift_date'].nunique()
            if worked_days > max_days:
                errors.append(f"Employee {emp} exceeds max work days in week {week}: {worked_days} > {max_days}")

    ## 5) Contract hours on average across all weeks
    num_weeks = assignments_df['week'].nunique()
    for emp, group in assignments_df.groupby("employee_id"):
        cap_minutes = int(workers.loc[workers['medewerker_id'] == emp, 'contract_minutes'].iloc[0])
        total_minutes = group['duration_min'].sum()
        allowed_total = cap_minutes * num_weeks
        if total_minutes > allowed_total:
            errors.append(
                f"Employee {emp} exceeds average contract hours: "
                f"{total_minutes} min worked > {allowed_total} min allowed over {num_weeks} weeks"
            )

    ## 6) No shifts overlapping with unavailable times
    for _, r in onb.iterrows():
        emp = r['Medewerker id']
        if emp not in workers['medewerker_id'].values:
            continue

        besch = r['Beschikbaarheid'].lower()
        if besch == 'beschikbaar':
            continue

        date_unb = r['Datum']
        start_unb = r.get('Beschikbaarheid_tijd_vanaf')
        end_unb = r.get('Beschikbaarheid_tijd_tm')

        emp_shifts = assignments_df[assignments_df["employee_id"] == emp]
        for _, sr in emp_shifts.iterrows():
            if sr['shift_date'] != date_unb:
                continue

            if pd.isna(start_unb) or pd.isna(end_unb):
                errors.append(f"Employee {emp} scheduled on fully unavailable day {date_unb}, shift {sr['shift_id']}.")
                continue

            overlap = not (sr['end_time'] <= start_unb or sr['start_time'] >= end_unb)
            if overlap:
                errors.append(
                    f"Employee {emp} scheduled during unavailable time on {date_unb}, "
                    f"shift {sr['shift_id']} ({sr['start_time']}–{sr['end_time']} overlaps with {start_unb}–{end_unb})."
                )

    ## 7.1) Max 5 consecutive nights
    for emp, group in assignments_df.groupby("employee_id"):
        emp_nights = group[group['is_night']].sort_values("shift_date")
        dates = emp_nights['shift_date'].tolist()
        max_consec = 7 if emp in COA_7_1_exempt else 5
        for i in range(len(dates) - max_consec):
            window = dates[i:i+max_consec+1]
            if (window[-1] - window[0]).days == max_consec:
                errors.append(f"Employee {emp} works >{max_consec} consecutive nights: {window}")

    ## 7.2) After ≥3 consecutive nights → 46h rest (next 2 days off)
    for emp, group in assignments_df.groupby("employee_id"):
        emp_nights = group[group['is_night']].sort_values("shift_date")
        night_dates = emp_nights["shift_date"].tolist()

        for i in range(len(night_dates) - 2):
            d0, d1, d2 = night_dates[i:i+3]
            if d1 == d0 + timedelta(days=1) and d2 == d1 + timedelta(days=1):
                next_day = d2 + timedelta(days=1)
                if next_day in night_dates:  # still part of a longer block
                    continue
                blocked_days = [next_day, next_day + timedelta(days=1)]
                next_shifts = group[group["shift_date"].isin(blocked_days)]
                if len(next_shifts) > 0:
                    errors.append(
                        f"Employee {emp} has shifts {next_shifts['shift_id'].tolist()} within 46h rest after nights {d0}, {d1}, {d2}."
                    )

    ## 7.3) Max 35 nights per 13 weeks
    for emp, group in assignments_df.groupby("employee_id"):
        emp_nights = group[group['is_night']]
        for start_week in range(shifts['week'].min(), shifts['week'].max()-12):
            window = emp_nights[(emp_nights['week'] >= start_week) & (emp_nights['week'] < start_week+13)]
            if len(window) > 35:
                errors.append(f"Employee {emp} has {len(window)} nights in weeks {start_week}-{start_week+12} (> 35).")

    ## 7.4) Age > 55: no night shifts
    for emp, group in assignments_df.groupby("employee_id"):
        if emp in CAO_7_4_exempt:
            continue
        leeftijd = int(workers.loc[workers['medewerker_id'] == emp, 'leeftijd'].iloc[0])
        if leeftijd > 55 and group['is_night'].any():
            errors.append(f"Employee {emp} (age {leeftijd}) assigned to night shifts: {group[group['is_night']==True]['shift_id'].tolist()}")

    ## 8) Qualification match
    for _, row in assignments_df.iterrows():
        sid = row['shift_id']
        required_quals = shifts_lookup.loc[sid, 'qualification']
        if not isinstance(required_quals, list):
            required_quals = [required_quals]
        emp_quals = workers.loc[workers['medewerker_id'] == row['employee_id'], 'deskundigheid'].iloc[0]
        if not any(q in emp_quals for q in required_quals):
            errors.append(f"Employee {row['employee_id']} lacks qualification {required_quals} for shift {sid}.")

    ## 9) Employee-specific constraints
    for _, row in assignments_df.iterrows():
        emp = row['employee_id']
        sid = row['shift_id']
        shift = shifts_lookup.loc[sid]
        date = row['shift_date']
        day_of_week = int(shift['day_of_week'])
        is_night = shift['is_night']

        # 9.1) Hennie: only night shifts
        if emp == '2097' and not is_night:
            errors.append(f"Employee 2097 (Hennie) assigned to non-night shift {sid} on {date}.")

        # 9.2) Romy: weekends only
        if emp == '2653' and day_of_week not in [5, 6]:
            errors.append(f"Employee 2653 (Romy) assigned to weekday shift {sid} on {date}.")

        # 9.3) Jasmijn: Friday night + weekends
        if emp == '3888':
            allowed = (day_of_week == 4 and is_night) or day_of_week in [5, 6]
            if not allowed:
                errors.append(f"Employee 3888 (Jasmijn) assigned to disallowed shift {sid} on {date}.")

        # 9.4) Maike: no Mon/Tue/Wed
        if emp == '601011' and day_of_week in [0, 1, 2]:
            errors.append(f"Employee 601011 (Maike) assigned to shift {sid} on {date} (Mon/Tue/Wed not allowed).")

        # 9.5) Marlies: Max 2 days in a row, then 2 days off
        if emp == '603722':
            days = assignments_df[assignments_df['employee_id'] == emp]['shift_date'].sort_values().tolist()
            blocks, block = [], []
            for d in days:
                if not block or d == block[-1] + timedelta(days=1):
                    block.append(d)
                else:
                    blocks.append(block)
                    block = [d]
            if block:
                blocks.append(block)
            for b in blocks:
                if len(b) > 2:
                    errors.append(f"Employee 603722 (Marlies) works >2 days in a row: {b}.")
                next_two_days = [b[-1] + timedelta(days=1), b[-1] + timedelta(days=2)]
                next_shifts = assignments_df[(assignments_df['employee_id'] == emp) & (assignments_df['shift_date'].isin(next_two_days))]
                if len(next_shifts) > 0:
                    errors.append(f"Employee 603722 (Marlies) does not have 2 days off after block {b}.")

        # 9.6) Teuna: 7-night / 7-off pattern
        if emp == '602859':
            nights = assignments_df[(assignments_df['employee_id'] == emp) & (assignments_df['is_night'])]['shift_date'].sort_values().tolist()
            blocks, block = [], []
            for d in nights:
                if not block or d == block[-1] + timedelta(days=1):
                    block.append(d)
                else:
                    blocks.append(block)
                    block = [d]
            if block:
                blocks.append(block)
            for i, b in enumerate(blocks):
                if i not in [0, len(blocks)-1] and len(b) != 7:
                    errors.append(f"Employee 602859 (Teuna) has night block of length {len(b)} (expected 7): {b}.")
                following = [b[-1] + timedelta(days=i+1) for i in range(7)]
                following = [d for d in following if d in assignments_df['shift_date'].values]
                next_shifts = assignments_df[(assignments_df['employee_id'] == emp) & (assignments_df['shift_date'].isin(following))]
                if len(next_shifts) > 0:
                    errors.append(f"Employee 602859 (Teuna) did not get 7 days off after block {b}.")

        # 9.7) Maria: Max 3 shifts per week
        if emp == '627311':
            emp_group = assignments_df[assignments_df['employee_id'] == emp]
            for week, week_group in emp_group.groupby("week"):
                if len(week_group) > 3:
                    errors.append(f"Employee 627311 (Maria) has {len(week_group)} shifts in week {week} (> 3).")

        # 9.8) Carolien: Max 3 shifts per week + only late or night
        if emp == '602056':
            emp_group = assignments_df[assignments_df['employee_id'] == emp]
            for week, week_group in emp_group.groupby("week"):
                if len(week_group) > 3:
                    errors.append(f"Employee 602056 (Carolien) has {len(week_group)} shifts in week {week} (> 3).")
                for _, shift_row in week_group.iterrows():
                    shift_name = shifts_lookup.loc[shift_row['shift_id'], 'shift_name']
                    if 'A' not in shift_name and not shift_row['is_night']:
                        errors.append(f"Employee 602056 (Carolien) assigned to non-late shift {shift_row['shift_id']} ({shift_name}).")

    if errors:
        print(f"=== VALIDATION FAILED: {len(errors)} issue(s) ===")
        for e in errors:
            print("❌", e)
    else:
        print("✅ All constraints satisfied!")


In [183]:
validate_auto_rooster(result["assignments_df"], 
                                                     preprocessed_data['shifts'],
                                                        preprocessed_data['workers'],
                                                        preprocessed_data['dur_min'],
                                                        preprocessed_data['onb'],
                                                        preprocessed_data['night_shifts'],
                                                        preprocessed_data['night_shifts_by_week'])

✅ All constraints satisfied!
